In [ ]:
import os
import time
print(1)
while not os.path.isfile('file.txt'):
    time.sleep(1)
print(1)


1
1


In [ ]:
!unrar x /content/drive/MyDrive/master

Выходные данные были обрезаны до нескольких последних строк (5000).
Extracting  master/PZZ/divided/processed/cloud/2017_13_5_3.jpg            38%  OK 
Extracting  master/PZZ/divided/processed/cloud/2017_13_5_4.jpg            38%  OK 
Extracting  master/PZZ/divided/processed/cloud/2017_13_6_1.jpg            38%  OK 
Extracting  master/PZZ/divided/processed/cloud/2017_13_6_2.jpg            38%  OK 
Extracting  master/PZZ/divided/processed/cloud/2017_13_6_3.jpg            38%  OK 
Extracting  master/PZZ/divided/processed/cloud/2017_13_6_4.jpg            38%  OK 
Extracting  master/PZZ/divided/processed/cloud/2017_13_7_1.jpg            38%  OK 
Extracting  master/PZZ/divided/processed/cloud/2017_13_7_2.jpg            38%  OK 
Extracting  master/PZZ/divided/processed/cloud/2017_13_7_3.jpg            39%  OK 
Extracting  master/PZZ/divided/processed/cloud/2017_13_7_4.jpg            39%  OK 
Extracting  

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install git+https://github.com/qubvel/segmentation_models

  Cloning https://github.com/qubvel/segmentation_models to /tmp/pip-req-build-0x0vk_ev
  Running command git clone --filter=blob:none --quiet https://github.com/qubvel/segmentation_models /tmp/pip-req-build-0x0vk_ev
  Resolved https://github.com/qubvel/segmentation_models to commit e951c6747f75fa9e7240816d1c79dd2e66813123
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 1.4 MB/s eta 0:00:00
  Created wheel for segmentation-models: filename=segmentation_models-1.0.1-py3-none-any.whl size=33791 sha256=f3eb5d18fe0d256d541c73db2fb45437d7c419a102ddf4c11c28a538fa69ec61
  Stored in directory: /tmp/pip-ephem-wheel-cache-080k6ozi/wheels/ce/d6/f1/5d00e82b3893c5f1ffee43bf7b8877148af09c7c9c6c4882c9
Successfully built segmentation-models


In [ ]:
import shutil
shutil.copytree('/content/drive/MyDrive/38-Cloud_training', '38-Cloud_training')
import os
path = '/content/drive/MyDrive/sparcs_data_L8'
shutil.copytree(path, 'sparcs')

'sparcs'

In [ ]:
import numpy as np
import os
os.environ["SM_FRAMEWORK"] = "tf.keras"

from tensorflow import keras
import segmentation_models as sm
import segmentation_models as sm
from tensorflow import keras
import cv2
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.models import load_model
import tifffile
from tifffile import imread
from skimage.transform import resize

Segmentation Models: using `tf.keras` framework.


In [ ]:
a = np.array([[[1, 2, 3], [4, 5, 6], [7, 8, 9]], [[17, 11, 19], [13, 14, 15], [16, 17, 18]]])
print(np.min(a, axis = 2))

[[ 1  4  7]
 [11 13 16]]


In [ ]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))
physical_devices = tf.config.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(physical_devices[0], True)
#tf.config.experimental.set_memory_growth(physical_devices[1], True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
keras.backend.set_image_data_format('channels_last')
devices = tf.config.experimental.list_physical_devices("GPU")
print(devices)
print(tf.distribute.HierarchicalCopyAllReduce())
mirrored_strategy = tf.distribute.MirroredStrategy(cross_device_ops = tf.distribute.HierarchicalCopyAllReduce())

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
a = [0] * 12
a[0] = [0, 0, 0]
a[1] = [0, 0, 255]
a[2] = [0, 255, 0]
a[3] = [255, 0, 0]
a[4] = [255, 255, 0]
a[5] = [255, 0, 255]
a[6] = [0, 255, 255]
a[8] = [255, 255, 255]
a[7] = [179, 73, 156]
a[9] = [1, 1, 0.5]
a[10] = [0.5, 1, 0.5]
a[11] = [0.5, 1, 1]

In [ ]:
x = np.zeros((256, 256, 3))
for i in range(256):
    for j in range(256):
        x[i, j] = a[9]
y = cv2.imwrite('example.jpg', x * 255)
print(y)

True


In [ ]:
def convert_res(res):
    res1 = np.argmax(res, axis = 3)
    ans = np.empty((res.shape[0], res.shape[1], res.shape[2], 3))
    #print(res1.shape)
    for j in range(12):
        x = np.nonzero((res1 == j))
        #print(x)
        ans[x[:][0], x[:][1], x[:][2]] = a[j]
    return ans

In [ ]:
import numpy as np
from PIL import Image
print('*'*30)
print('Loading and preprocessing train data...')
print('*'*30)

def train_data():
        path = 'home/u39710/divided'
        list = os.listdir(path + '/train')
        listimg = list
        l1 = np.random.permutation(listimg)
        for j in range(len(l1)):
            #for k in range(bs):
                i = l1[j]
                #print(path)
                img = Image.open(os.path.join(path, 'train', i))
                img = img.resize((256, 256))
                img_train = np.asarray(img).astype('float16') / 255

                img = Image.open(os.path.join(path, 'pzz', i)).resize((256, 256))
                #img = img.resize((2048, 2048))
                mask_train = np.asarray(img)
                mask_train1 = mask_train
                if np.count_nonzero(mask_train1==0) < 256*256:
                    m1 = np.zeros((256, 256, 9))
                    for k in range(0, 9):
                                  if np.argwhere(mask_train1 == k).shape[0] != 0:
                                      x = np.argwhere(mask_train1 == k)
                                      m1[x[:, 0], x[:, 1] , k] = 1
                              #mask_train = 1 - mask_train
                    m1[:, :, 0] += m1[:, :, 8]
                    if np.sum(m1[:, :, 0]) < 256 * 256 * 0.75:
                        yield img_train, m1[:, :, :8]
                #yield tf.convert_to_tensor(img_train), tf.convert_to_tensor(mask_train)


def valid_data():
        path = 'home/u39710/divided'
        list = os.listdir(path + '/valid')
        listimg = list
        l1 = np.random.permutation(listimg)
        for j in range(len(l1)):
            #for k in range(bs):
                i = l1[j]
                img = Image.open(os.path.join(path, 'valid', i))
                img = img.resize((256, 256))
                img_train = np.asarray(img).astype('float16') / 255

                img = Image.open(os.path.join(path, 'pzz', i)).resize((256, 256))
                #img = img.resize((2048, 2048))
                mask_train = np.asarray(img)
                mask_train1 = mask_train
                if np.count_nonzero(mask_train1==0) < 256*256:
                    m1 = np.zeros((256, 256, 9))
                    for k in range(0, 9):
                                  if np.argwhere(mask_train1 == k).shape[0] != 0:
                                      x = np.argwhere(mask_train1 == k)
                                      m1[x[:, 0], x[:, 1] , k] = 1
                              #mask_train = 1 - mask_train

                    m1[:, :, 0] += m1[:, :, 8]
                    if np.sum(m1[:, :, 0]) < 256 * 256 * 0.75:
                        yield img_train, m1[:, :, :8]
                #yield tf.convert_to_tensor(img_train), tf.convert_to_tensor(mask_train)

def test_data(data):
    def train():
        path = 'D:/Misha/master/images/divided'
        list = os.listdir(path + '/test')
        l1 = list
        for j in range(len(l1)):
            #for k in range(bs):
                i = l1[j]
                img = Image.open(os.path.join(path, 'test', i))
                img = img.resize((2048, 2048))
                img_train = np.asarray(img).astype('float16') / 255

                img = Image.open(os.path.join('D:/Misha/master/images/PZZ/old', i))
                img = img.resize((2048, 2048))
                m1 = np.zeros((2048, 2048, 12))
                mask_train = np.asarray(img).astype('float16')
                for i in range((mask_train.shape[0] + 511) // 512):
                    for j in range((mask_train.shape[1] + 511) // 512):
                        mask_train1 = mask_train[i * 512 : (i + 1) * 512, j * 512 : (j + 1) * 512]
                        img_train1 = img_train[i * 512 : (i + 1) * 512, j * 512 : (j + 1) * 512]

                        for k in range(0, 12):
                            if np.argwhere(mask_train1 == k).shape[0] != 0:
                                x = np.argwhere(mask_train1 == k)
                                m1[x[:, 0], x[:, 1] , k] = 1
                yield tf.convert_to_tensor(img_train), tf.convert_to_tensor(m1)
                #yield tf.convert_to_tensor(img_train), tf.convert_to_tensor(mask_train)
    return train

print('*'*30)
print('Creating and compiling model...')
print('*'*30)
strategy = tf.distribute.MirroredStrategy(cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))


******************************
Loading and preprocessing train data...
******************************
******************************
Creating and compiling model...
******************************
Number of devices: 1


In [ ]:
count = [0] * 9
for i, j in train_data():
  for k in range(9):
    count[k] += np.count_nonzero(j[:, :, k])
  print(count)

[63131, 255, 172, 121, 74, 148, 208, 341, 414]
[87250, 2427, 1588, 1061, 892, 1475, 5760, 7677, 15527]
[97817, 5871, 6477, 3131, 4904, 5009, 12446, 16399, 31531]
[114781, 7619, 7727, 3983, 5655, 6230, 14779, 20776, 60353]
[175705, 8034, 8033, 4173, 5780, 6431, 15106, 21279, 61794]
[224329, 9280, 8939, 4707, 6184, 7104, 16324, 23461, 67667]
[274557, 10458, 9700, 5178, 6568, 7640, 17379, 25300, 73400]
[330000, 11147, 10197, 5470, 6798, 7983, 17995, 26475, 77592]
[386941, 11886, 10691, 5797, 7051, 8330, 18680, 27596, 80056]
[397267, 13338, 11521, 6616, 9521, 10854, 39603, 45602, 86492]
[448828, 14655, 12356, 7177, 9975, 11463, 40838, 47560, 89748]
[489203, 15382, 12759, 7441, 10674, 12288, 52428, 55911, 91037]
[541152, 16735, 13666, 7980, 11092, 12923, 53519, 57678, 94373]
[595589, 18631, 14253, 8257, 11345, 13278, 54228, 58893, 98036]
[639584, 24455, 14966, 8701, 12439, 14709, 62016, 62312, 98724]
[672581, 25303, 16343, 9261, 13923, 16641, 80482, 66190, 101064]
[736856, 25434, 16429, 930

In [ ]:
print(count[8] / sum(count))

0.13708755651562668


In [ ]:
path = 'D:/sha/master/cloud/mask'
l = os.listdir(path)
for i in l:
    if i[-1] == 'g':
        img = Image.open(os.path.join(path, i))
        img = np.array(img) / 255
        im1 = np.zeros((img.shape[0], img.shape[1]))
        im1[np.nonzero(np.sum(img, axis = 2) > 1)] = 1
        cv2.imwrite(os.path.join(path, 'new', i), im1 * 255)

In [ ]:
    import sklearn
    import sklearn.model_selection
    path = 'D:/Misha/master/cloud'
    l = os.listdir(os.path.join(path, 'cloud'))
    train, valid = sklearn.model_selection.train_test_split(l, test_size = 0.2)
    train, test = sklearn.model_selection.train_test_split(train, test_size = 0.2)
    for i in valid:
            os.replace(os.path.join(path, 'mask', i), os.path.join(path,'valid', 'mask', i))
            os.replace(os.path.join(path, 'cloud', i), os.path.join(path,'valid', 'cloud', i))

FileNotFoundError: ignored

In [ ]:
for i in test:
            os.replace(os.path.join(path, 'mask', i), os.path.join(path,'test', 'mask', i))
            os.replace(os.path.join(path, 'cloud', i), os.path.join(path,'test', 'cloud', i))
for i in train:
            os.replace(os.path.join(path, 'mask', i), os.path.join(path,'train', 'mask', i))
            os.replace(os.path.join(path, 'cloud', i), os.path.join(path,'train', 'cloud', i))

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split

l = [i for i in os.listdir('sparcs') if i[-8:] == 'data.tif']
train, valid = train_test_split(l, test_size = 0.2)
os.mkdir('sparcs/train')
for i in train:
  os.replace(os.path.join('sparcs', i), os.path.join('sparcs/train', i))
os.mkdir('sparcs/valid')
for i in valid:
  os.replace(os.path.join('sparcs', i), os.path.join('sparcs/valid', i))

In [ ]:
import numpy as np
from PIL import Image
print('*'*30)
print('Loading and preprocessing train data...')
print('*'*30)

def train1():
        size = 384
        data = 'images'
        path = '38-Cloud_training'
        list = os.listdir(path + '/images/train')
        listimg = list
        l1 = np.random.permutation(listimg)
        for j in range(len(l1)):
            #for k in range(bs):
                i = l1[j]
                #print(path)
                img = Image.open(os.path.join(path, 'images', 'train', i))
                img = img.resize((256, 256))
                img_train = np.asarray(img).astype('float16') / 255
                if os.path.exists(os.path.join(path, 'labels', i[:-3] + 'png')):
                    img = np.array(Image.open(os.path.join(path, 'labels', i[:-3] + 'png')).resize((256, 256)))
                else:
                    img = np.array(Image.open(os.path.join(path, 'labels', i[:-3] + 'TIF')).resize((256, 256)))
               #img = img.resize((2048, 2048))
                mask_train = np.asarray(img).astype('float16').reshape((256, 256, 1)) / 255
                #print(np.max(mask_train))
                 #mask_train = 1 - mask_train
                yield tf.convert_to_tensor(img_train), tf.convert_to_tensor(mask_train)
                #yield tf.convert_to_tensor(img_train), tf.convert_to_tensor(mask_train)
        path = 'divided'
        list = [i for i in os.listdir(path + '/train') if i[-5] == '1']
        listimg = list
        l1 = np.random.permutation(listimg)
        for j in range(len(l1)):
            #for k in range(bs):
                i = l1[j]
                #print(path)
                #img_train = resize(imread(os.path.join(path, 'train', i)), (1024, 1024, 10))
                #img_train = np.asarray(img).astype('float16') / 255

                img_train = np.array(Image.open(os.path.join(path, 'train', i)).resize((256, 256)))
                img = np.zeros((256, 256, 1))
                yield tf.convert_to_tensor(img_train), tf.convert_to_tensor(img)


def valid1():
        size = 384
        data = 'images'
        path = '38-Cloud_training'
        list = os.listdir(path + '/images/valid')
        listimg = list
        l1 = np.random.permutation(listimg)
        for j in range(len(l1)):
            #for k in range(bs):
                i = l1[j]
                #print(path)
                img = Image.open(os.path.join(path,'images', 'valid', i))
                img = img.resize((256, 256))
                img_train = np.asarray(img).astype('float16') / 255

                if os.path.exists(os.path.join(path, 'labels', i[:-3] + 'png')):
                    img = np.array(Image.open(os.path.join(path, 'labels', i[:-3] + 'png')).resize((256, 256)))
                else:
                    img = np.array(Image.open(os.path.join(path, 'labels', i[:-3] + 'TIF')).resize((256, 256)))
                #img = img.resize((2048, 2048))
                mask_train = np.asarray(img).astype('float16').reshape((256, 256, 1)) / 255
                yield tf.convert_to_tensor(img_train), tf.convert_to_tensor(mask_train)
                #yield tf.convert_to_tensor(img_train), tf.convert_to_tensor(mask_train)
        path = 'divided'
        list = [i for i in os.listdir(path + '/valid') if i[-5] == '1']
        listimg = list
        l1 = np.random.permutation(listimg)
        for j in range(len(l1)):
            #for k in range(bs):
                i = l1[j]
                #print(path)
                #img_train = resize(imread(os.path.join(path, 'train', i)), (1024, 1024, 10))
                #img_train = np.asarray(img).astype('float16') / 255

                img_train = np.array(Image.open(os.path.join(path, 'valid', i)).resize((256, 256)))
                img = np.zeros((256, 256, 1))
                yield tf.convert_to_tensor(img_train), tf.convert_to_tensor(img)


def test_data(data):
    def train():
        path = 'D:/Misha/master/new/' + data
        list = os.listdir(path + '/test')
        l1 = list
        for j in range(len(l1)):
            #for k in range(bs):
                i = l1[j]
                img = Image.open(os.path.join(path, 'test', i))
                img = img.resize((2048, 2048))
                img_train = np.asarray(img).astype('float16') / 255

                img = Image.open(os.path.join('D:/Misha/master/new/labels', i))
                img = img.resize((2048, 2048))
                m1 = np.zeros((2048, 2048, 12))
                mask_train = np.asarray(img).astype('float16')
                for i in range((mask_train.shape[0] + 511) // 512):
                    for j in range((mask_train.shape[1] + 511) // 512):
                        mask_train1 = mask_train[i * 512 : (i + 1) * 512, j * 512 : (j + 1) * 512]
                        img_train1 = img_train[i * 512 : (i + 1) * 512, j * 512 : (j + 1) * 512]

                        for k in range(0, 12):
                            if np.argwhere(mask_train1 == k).shape[0] != 0:
                                x = np.argwhere(mask_train1 == k)
                                m1[x[:, 0], x[:, 1] , k] = 1
                yield tf.convert_to_tensor(img_train), tf.convert_to_tensor(m1)
                #yield tf.convert_to_tensor(img_train), tf.convert_to_tensor(mask_train)
    return train

print('*'*30)
print('Creating and compiling model...')
print('*'*30)
strategy = tf.distribute.MirroredStrategy(cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))


******************************
Loading and preprocessing train data...
******************************
******************************
Creating and compiling model...
******************************
Number of devices: 1


In [ ]:
def f1_loss(y_true, y_pred):

    tp = y_true*y_pred
    tn = (1-y_true)*(1-y_pred)
    fp = (1-y_true)*y_pred
    fn = y_true*(1-y_pred)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    return 1 - f1

In [ ]:
for i, j in train1():
  print(np.min(j), np.max(j))

pidor1
0.0 0.0
0.0 0.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 0.0
0.0 1.0
1.0 1.0
0.0 1.0
0.0 0.0
1.0 1.0
1.0 1.0
0.0 1.0
0.0 1.0
1.0 1.0
0.0 0.0
0.0 1.0
0.0 0.0
0.0 1.0
0.0 0.4275
0.0 0.0
0.0 0.0
0.0 0.0
0.0 1.0
0.0 1.0
0.612 1.0
1.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 0.0
0.0 1.0
0.0 0.0
1.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
1.0 1.0
0.0 1.0
1.0 1.0
0.0 0.0
0.0 0.0
0.0 1.0
0.0 1.0
0.0 0.0
0.0 1.0
0.3215 1.0
0.0 1.0
1.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 0.0
0.0 0.2354
0.0 0.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 1.0
1.0 1.0
0.0 0.0
0.0 0.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
1.0 1.0
1.0 1.0
0.0 0.0
0.0 1.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 1.0
0.0 1.0
0.0 0.0
0.0 0.0
0.0 1.0
0.0 1.0
0.0 0.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.02353 1.0
0.0 1.0
0.0 0.0
0.0 1.0
0.0 1.0
0.0 0.0
0.0 1.0
0.0 1.0
0.0 0.0
0.0 1.0
0.0 0.0
0.0 1.0
0.0 0.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 0.0
1.0 1.0
0.0 1.0
0.0 0.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 0.0
0.0 0.0
0.0 1.0
0.0 0.0
0.0 1.0
0.0 0.0
0.0 1.0
0.0 0.0
0.

KeyboardInterrupt: ignored

In [ ]:
import keras.backend as K
#'PSPNet' : sm.PSPNet,
models = {'Linknet':sm.Linknet, 'FPN':sm.FPN, 'Unet': sm.Unet}
val_f1 = {'Unet': 0, 'FPN' : 0, 'Linknet' : 0}

for model_str in models:
    #print(model_str)
    size = 512
    with strategy.scope():
        model = models[model_str](classes = 1, activation = 'sigmoid')
        model.compile(optimizer = 'adam', loss = sm.losses.dice_loss, metrics = ['accuracy', sm.metrics.f1_score])
    print('*'*30)
    print('Fitting model ' + model_str + '...')
    print('*'*30)
    s = 'images'
    size = 256
    train = tf.data.Dataset.from_generator(train1, output_signature=(
             tf.TensorSpec(shape=(size, size, 3), dtype=tf.float16),
             tf.TensorSpec(shape=(size, size, 1), dtype=tf.float16))).batch(12)

    valid = tf.data.Dataset.from_generator(valid1, output_signature=(
             tf.TensorSpec(shape=(size, size, 3), dtype=tf.float16),
             tf.TensorSpec(shape=(size, size, 1), dtype=tf.float16))).batch(12)

    epochs = 3
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10)
    model_checkpoint = ModelCheckpoint(f'drive/MyDrive/weights_' + model_str + f'_buildings_{epochs}epochs.tf', monitor='val_accuracy', save_best_only=True, mode = 'max')

    history =  model.fit(train, epochs=epochs, verbose=1,
              validation_data = valid,
              callbacks=[model_checkpoint, early_stopping])

    val_f1[model_str] = np.max(np.asarray(history.history['val_f1-score']))
    print(val_f1)
    """if model_str != 'PSPNet':
        path = 'D:/Misha/master/new'
        test = tf.data.Dataset.from_generator(test_data('sr'), output_signature=(
                 tf.TensorSpec(shape=(1024, 1024, 3), dtype=tf.float16),
                 tf.TensorSpec(shape=(1024, 1024, 12), dtype=tf.float16))).batch(1)
        res = model.predict(test)
        res = convert_res(res)
        list = os.listdir(path + '/sr/test')
        for i in range(len(list)):

            x = cv2.imwrite(os.path.join(path, 'predicted', list[i][:-4] + '_' + model_str + '.jpg'), (res[i] * 255))
            if not x:
                print(i)"""

******************************
Fitting model Linknet...
******************************
Epoch 1/3
pidor1
    564/Unknown - 179s 282ms/step - loss: 0.2768 - accuracy: 0.8140 - f1-score: 0.7232pidor1


565/565 [==============================] - 239s 387ms/step - loss: 0.2767 - accuracy: 0.8140 - f1-score: 0.7237 - val_loss: 0.2750 - val_accuracy: 0.8051 - val_f1-score: 0.7242
Epoch 2/3
pidor1
564/565 [============================>.] - ETA: 0s - loss: 0.2435 - accuracy: 0.8375 - f1-score: 0.7565pidor1


565/565 [==============================] - 213s 377ms/step - loss: 0.2435 - accuracy: 0.8375 - f1-score: 0.7562 - val_loss: 0.2567 - val_accuracy: 0.8518 - val_f1-score: 0.7439
Epoch 3/3
pidor1
565/565 [==============================] - 194s 343ms/step - loss: 0.2025 - accuracy: 0.8647 - f1-score: 0.7974 - val_loss: 0.5442 - val_accuracy: 0.3014 - val_f1-score: 0.4553
{'Unet': 0, 'FPN': 0, 'Linknet': 0.8518492579460144}
******************************
Fitting model FPN...
******************************
Epoch 1/3
pidor1
    565/Unknown - 205s 333ms/step - loss: 0.3393 - accuracy: 0.7342 - f1-score: 0.6608pidor1


565/565 [==============================] - 264s 438ms/step - loss: 0.3393 - accuracy: 0.7342 - f1-score: 0.6608 - val_loss: 0.9008 - val_accuracy: 0.6931 - val_f1-score: 0.0990
Epoch 2/3
pidor1
565/565 [==============================] - ETA: 0s - loss: 0.3107 - accuracy: 0.7776 - f1-score: 0.6897pidor1


565/565 [==============================] - 251s 444ms/step - loss: 0.3107 - accuracy: 0.7776 - f1-score: 0.6897 - val_loss: 0.8858 - val_accuracy: 0.6954 - val_f1-score: 0.1138
Epoch 3/3
pidor1
565/565 [==============================] - ETA: 0s - loss: 0.2926 - accuracy: 0.7941 - f1-score: 0.7078pidor1


565/565 [==============================] - 251s 444ms/step - loss: 0.2926 - accuracy: 0.7941 - f1-score: 0.7078 - val_loss: 0.2977 - val_accuracy: 0.7898 - val_f1-score: 0.7025
{'Unet': 0, 'FPN': 0.7898324728012085, 'Linknet': 0.8518492579460144}
******************************
Fitting model Unet...
******************************
Epoch 1/3
pidor1
    565/Unknown - 171s 274ms/step - loss: 0.2925 - accuracy: 0.7976 - f1-score: 0.7079pidor1


565/565 [==============================] - 225s 369ms/step - loss: 0.2925 - accuracy: 0.7976 - f1-score: 0.7079 - val_loss: 0.5444 - val_accuracy: 0.2920 - val_f1-score: 0.4564
Epoch 2/3
pidor1
564/565 [============================>.] - ETA: 0s - loss: 0.2767 - accuracy: 0.8061 - f1-score: 0.7233pidor1


565/565 [==============================] - 204s 360ms/step - loss: 0.2767 - accuracy: 0.8060 - f1-score: 0.7233 - val_loss: 0.3391 - val_accuracy: 0.7081 - val_f1-score: 0.6604
Epoch 3/3
pidor1
565/565 [==============================] - 193s 341ms/step - loss: 0.2620 - accuracy: 0.8201 - f1-score: 0.7383 - val_loss: 0.5436 - val_accuracy: 0.2926 - val_f1-score: 0.4578
{'Unet': 0.7081100344657898, 'FPN': 0.7898324728012085, 'Linknet': 0.8518492579460144}


In [ ]:
    """res1 = np.argmax(res, axis = 3)
    ans = np.empty((res.shape[0], res.shape[1], res.shape[2], 3))
    print(np.min(res1), np.max(res1))
    print(res1.shape)
    print(np.sum(res1))
    for j in range(12):
        x = np.nonzero((res1 == j))
        print(x)
        ans[x[:][0], x[:][1], x[:][2]] = a[j]
        print(len(x))
    #res = convert_res(res)
    print(np.sum(ans))
    res = ans
    list = os.listdir(path + '/sr/test')
    for i in range(len(list)):

            x = cv2.imwrite(os.path.join(path, 'predicted', list[i][:-4] + '_' + model_str + '.jpg'), (res[i] * 255))
            if not x:
                print(i)
    print(res.shape)
    print(np.sum(res[0]))
    print(res[0])"""

"res1 = np.argmax(res, axis = 3)\nans = np.empty((res.shape[0], res.shape[1], res.shape[2], 3))\nprint(np.min(res1), np.max(res1))\nprint(res1.shape)\nprint(np.sum(res1))\nfor j in range(12):\n    x = np.nonzero((res1 == j))\n    print(x)\n    ans[x[:][0], x[:][1], x[:][2]] = a[j]\n    print(len(x))\n#res = convert_res(res)\nprint(np.sum(ans))\nres = ans\nlist = os.listdir(path + '\\sr\\test')\nfor i in range(len(list)):\n        \n        x = cv2.imwrite(os.path.join(path, 'predicted', list[i][:-4] + '_' + model_str + '.jpg'), (res[i] * 255))\n        if not x:\n            print(i)\nprint(res.shape)\nprint(np.sum(res[0]))\nprint(res[0])"

In [ ]:
res = 'Unet'
print(val_f1)
for i in models:
    if val_f1[i] > val_f1[res]:
        res = i
print(res)

{'Unet': 0.7081100344657898, 'FPN': 0.7898324728012085, 'Linknet': 0.8518492579460144}
Linknet


In [ ]:
res = 'Linknet'
res_l = 'focal_dice_loss'
losses = {'jaccard_loss' : sm.losses.jaccard_loss, 'dice_loss' : sm.losses.dice_loss, 'focal_loss' : sm.losses.categorical_focal_loss, 'crossentropy' : sm.losses.categorical_crossentropy, 'cce_dice_loss' : sm.losses.cce_dice_loss, 'bce_jaccard_loss' : sm.losses.cce_jaccard_loss, 'focal_dice_loss' : sm.losses.categorical_focal_dice_loss, 'focal_jaccard_loss' : sm.losses.categorical_focal_jaccard_loss}
models = {'Linknet':sm.Linknet, 'FPN':sm.FPN, 'Unet': sm.Unet}

In [ ]:
import keras.backend as K
model_str = res
#'PSPNet' : sm.PSPNet,
losses = {'jaccard_loss' : sm.losses.jaccard_loss, 'dice_loss' : sm.losses.dice_loss, 'focal_loss' : sm.losses.categorical_focal_loss, 'crossentropy' : sm.losses.categorical_crossentropy, 'cce_dice_loss' : sm.losses.cce_dice_loss, 'bce_jaccard_loss' : sm.losses.cce_jaccard_loss, 'focal_dice_loss' : sm.losses.categorical_focal_dice_loss, 'focal_jaccard_loss' : sm.losses.categorical_focal_jaccard_loss}
val_f1 = {'jaccard_loss':0, 'dice_loss':0, 'focal_loss':0, 'crossentropy':0, 'cce_dice_loss':0, 'bce_jaccard_loss':0, 'focal_dice_loss':0, 'focal_jaccard_loss':0}
def f1_loss (y_true, y_pred):

    tp = y_true*y_pred
    tn = (1-y_true)*(1-y_pred)
    fp = (1-y_true)*y_pred
    fn = y_true*(1-y_pred)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    return 1 - f1
for loss in losses:
    #print(model_str)
    size = 256
    with strategy.scope():
        model = models[res](classes = 1, activation = 'sigmoid')
        model.compile(optimizer = 'adam', loss = f1_loss, metrics = ['accuracy', sm.metrics.f1_score])
    print('*'*30)
    print('Fitting model ' + res + f' with {loss}...')
    print('*'*30)

    train = tf.data.Dataset.from_generator(train_data, output_signature=(
             tf.TensorSpec(shape=(size, size, 3), dtype=tf.float16),
             tf.TensorSpec(shape=(size, size, 9), dtype=tf.float16))).batch(12)

    valid = tf.data.Dataset.from_generator(valid_data, output_signature=(
             tf.TensorSpec(shape=(size, size, 3), dtype=tf.float16),
             tf.TensorSpec(shape=(size, size, 9), dtype=tf.float16))).batch(12)

    epochs = 3
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_f1-score', patience=10)
    model_checkpoint = ModelCheckpoint(f'drive/MyDrive/weights_' + res + f'_buildings_{epochs}epochs.tf', monitor='val_f1-score', save_best_only=True, mode = 'max')

    history =  model.fit(train, epochs=epochs, verbose=1,
              validation_data = valid,
              callbacks=[model_checkpoint, early_stopping])
    val_f1[loss] = np.max(np.asarray(history.history['val_f1-score']))


******************************
Fitting model Linknet with jaccard_loss...
******************************
Epoch 1/3
    124/Unknown - 95s 423ms/step - loss: 0.8976 - accuracy: 0.1113 - f1-score: 0.1582

124/124 [==============================] - 126s 673ms/step - loss: 0.8976 - accuracy: 0.1113 - f1-score: 0.1582 - val_loss: 0.9026 - val_accuracy: 0.1057 - val_f1-score: 0.1603
Epoch 2/3
 34/124 [=======>......................] - ETA: 34s - loss: 0.8945 - accuracy: 0.1059 - f1-score: 0.1586

KeyboardInterrupt: ignored

In [ ]:
res_l = 'focal_dice_loss'
print(val_f1)
for key in val_f1:
    if val_f1[res_l] < val_f1[key]:
        res_l = key
losses = {'jaccard_loss' : sm.losses.jaccard_loss, 'dice_loss' : sm.losses.dice_loss, 'focal_loss' : sm.losses.categorical_focal_loss, 'crossentropy' : sm.losses.categorical_crossentropy, 'cce_dice_loss' : sm.losses.cce_dice_loss, 'bce_jaccard_loss' : sm.losses.cce_jaccard_loss, 'focal_dice_loss' : sm.losses.categorical_focal_dice_loss, 'focal_jaccard_loss' : sm.losses.categorical_focal_jaccard_loss}

print(res)

{'jaccard_loss': 0.6831910610198975, 'dice_loss': 0.7074981927871704, 'focal_loss': 0.2919543385505676, 'crossentropy': 0.6422452926635742, 'cce_dice_loss': 0.6831895112991333, 'bce_jaccard_loss': 0.8402859568595886, 'focal_dice_loss': 0.8421272039413452, 'focal_jaccard_loss': 0.7311770915985107}
Linknet


In [ ]:
    #print(model_str)
    size = 256
    res = 'Linknet'
    rel_l = 'focal_dice_loss'
    epochs = 400
    with strategy.scope():
        model = load_model(f'drive/MyDrive/res_cloud_weights_' + res + f'_buildings_{epochs}epochs_var2.tf', compile = False)
        model.compile(optimizer = 'adam', loss = losses[res_l], metrics = ['accuracy', sm.metrics.f1_score])
    print('*'*30)
    print('Fitting model ' + res + f' with {res_l}...')
    print('*'*30)

    train = tf.data.Dataset.from_generator(train1, output_signature=(
             tf.TensorSpec(shape=(size, size, 3), dtype=tf.float16),
             tf.TensorSpec(shape=(size, size, 1), dtype=tf.float16))).batch(12)

    valid = tf.data.Dataset.from_generator(valid1, output_signature=(
             tf.TensorSpec(shape=(size, size, 3), dtype=tf.float16),
             tf.TensorSpec(shape=(size, size, 1), dtype=tf.float16))).batch(12)

    epochs = 400
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=100)
    model_checkpoint = ModelCheckpoint(f'drive/MyDrive/res_cloud_weights_' + res + f'_buildings_{epochs}epochs_var2.tf', monitor='val_accuracy', save_best_only=True, mode = 'max')

    history =  model.fit(train, epochs=epochs, verbose=1,
              validation_data = valid,
              callbacks=[model_checkpoint, early_stopping])


******************************
Fitting model Linknet with focal_dice_loss...
******************************
Epoch 1/400
pidor1
    601/Unknown - 327s 480ms/step - loss: 0.1600 - accuracy: 0.9169 - f1-score: 0.8520pidor1


601/601 [==============================] - 427s 646ms/step - loss: 0.1600 - accuracy: 0.9169 - f1-score: 0.8520 - val_loss: 0.5560 - val_accuracy: 0.3080 - val_f1-score: 0.4421
Epoch 2/400
pidor1
601/601 [==============================] - ETA: 0s - loss: 0.1549 - accuracy: 0.9215 - f1-score: 0.8569pidor1


601/601 [==============================] - 382s 635ms/step - loss: 0.1549 - accuracy: 0.9215 - f1-score: 0.8569 - val_loss: 0.5592 - val_accuracy: 0.3089 - val_f1-score: 0.4391
Epoch 3/400
pidor1
601/601 [==============================] - 335s 557ms/step - loss: 0.1480 - accuracy: 0.9164 - f1-score: 0.8630 - val_loss: 0.5572 - val_accuracy: 0.3080 - val_f1-score: 0.4410
Epoch 4/400
pidor1
601/601 [==============================] - 335s 558ms/step - loss: 0.1511 - accuracy: 0.9269 - f1-score: 0.8598 - val_loss: 0.5596 - val_accuracy: 0.3069 - val_f1-score: 0.4385
Epoch 5/400
pidor1
601/601 [==============================] - 367s 611ms/step - loss: 0.1561 - accuracy: 0.9221 - f1-score: 0.8551 - val_loss: 0.5558 - val_accuracy: 0.3073 - val_f1-score: 0.4423
Epoch 6/400
pidor1
601/601 [==============================] - ETA: 0s - loss: 0.1535 - accuracy: 0.9263 - f1-score: 0.8578pidor1


601/601 [==============================] - 349s 580ms/step - loss: 0.1535 - accuracy: 0.9263 - f1-score: 0.8578 - val_loss: 0.5571 - val_accuracy: 0.3151 - val_f1-score: 0.4413
Epoch 7/400
pidor1
601/601 [==============================] - ETA: 0s - loss: 0.1489 - accuracy: 0.9344 - f1-score: 0.8615pidor1


601/601 [==============================] - 349s 580ms/step - loss: 0.1489 - accuracy: 0.9344 - f1-score: 0.8615 - val_loss: 1.3874 - val_accuracy: 0.6849 - val_f1-score: 0.0089
Epoch 8/400
pidor1
601/601 [==============================] - 334s 556ms/step - loss: 0.1484 - accuracy: 0.9324 - f1-score: 0.8624 - val_loss: 0.5608 - val_accuracy: 0.3146 - val_f1-score: 0.4377
Epoch 9/400
pidor1
601/601 [==============================] - ETA: 0s - loss: 0.1475 - accuracy: 0.9392 - f1-score: 0.8626pidor1


...

601/601 [==============================] - 343s 571ms/step - loss: 0.1371 - accuracy: 0.9435 - f1-score: 0.8719 - val_loss: 0.9978 - val_accuracy: 0.6905 - val_f1-score: 0.1437
Epoch 25/400
pidor1
601/601 [==============================] - 365s 608ms/step - loss: 0.1376 - accuracy: 0.9294 - f1-score: 0.8715 - val_loss: 0.5556 - val_accuracy: 0.3089 - val_f1-score: 0.4425
Epoch 26/400
pidor1
601/601 [==============================] - 332s 552ms/step - loss: 0.1431 - accuracy: 0.9328 - f1-score: 0.8669 - val_loss: 0.5547 - val_accuracy: 0.3081 - val_f1-score: 0.4434
Epoch 27/400
pidor1
601/601 [==============================] - 330s 550ms/step - loss: 0.1378 - accuracy: 0.9352 - f1-score: 0.8716 - val_loss: 0.5542 - val_accuracy: 0.3086 - val_f1-score: 0.4439
Epoch 28/400
pidor1
601/601 [==============================] - 330s 549ms/step - loss: 0.1394 - accuracy: 0.9352 - f1-score: 0.8702 - val_loss: 0.5536 - val_accuracy: 0.3085 - val_f1-score: 0.4444
Epoch 29/400
pidor1
601/601 [======

601/601 [==============================] - 343s 570ms/step - loss: 0.1319 - accuracy: 0.9424 - f1-score: 0.8766 - val_loss: 0.8347 - val_accuracy: 0.6911 - val_f1-score: 0.2476
Epoch 39/400
pidor1
601/601 [==============================] - 330s 549ms/step - loss: 0.1371 - accuracy: 0.9399 - f1-score: 0.8716 - val_loss: 0.5635 - val_accuracy: 0.3219 - val_f1-score: 0.4378
Epoch 40/400
pidor1
601/601 [==============================] - 330s 549ms/step - loss: 0.1391 - accuracy: 0.9385 - f1-score: 0.8698 - val_loss: 0.5534 - val_accuracy: 0.3103 - val_f1-score: 0.4448
Epoch 41/400
pidor1
601/601 [==============================] - ETA: 0s - loss: 0.1309 - accuracy: 0.9435 - f1-score: 0.8773pidor1


601/601 [==============================] - 342s 568ms/step - loss: 0.1309 - accuracy: 0.9435 - f1-score: 0.8773 - val_loss: 0.9803 - val_accuracy: 0.6911 - val_f1-score: 0.1551
Epoch 42/400
pidor1
601/601 [==============================] - 329s 548ms/step - loss: 0.1332 - accuracy: 0.9420 - f1-score: 0.8753 - val_loss: 0.7017 - val_accuracy: 0.6911 - val_f1-score: 0.3340
Epoch 43/400
pidor1
601/601 [==============================] - 364s 606ms/step - loss: 0.1366 - accuracy: 0.9416 - f1-score: 0.8720 - val_loss: 1.1323 - val_accuracy: 0.6911 - val_f1-score: 0.0770
Epoch 44/400
pidor1
601/601 [==============================] - ETA: 0s - loss: 0.1291 - accuracy: 0.9427 - f1-score: 0.8789pidor1


601/601 [==============================] - 343s 571ms/step - loss: 0.1291 - accuracy: 0.9427 - f1-score: 0.8789 - val_loss: 0.6264 - val_accuracy: 0.7234 - val_f1-score: 0.3904
Epoch 45/400
pidor1
601/601 [==============================] - ETA: 0s - loss: 0.1296 - accuracy: 0.9435 - f1-score: 0.8787pidor1


601/601 [==============================] - 342s 570ms/step - loss: 0.1296 - accuracy: 0.9435 - f1-score: 0.8787 - val_loss: 0.6193 - val_accuracy: 0.9005 - val_f1-score: 0.3927
Epoch 46/400
pidor1
601/601 [==============================] - 330s 549ms/step - loss: 0.1308 - accuracy: 0.9437 - f1-score: 0.8772 - val_loss: 0.5881 - val_accuracy: 0.3206 - val_f1-score: 0.4143
Epoch 47/400
pidor1
601/601 [==============================] - 329s 548ms/step - loss: 0.1297 - accuracy: 0.9428 - f1-score: 0.8787 - val_loss: 0.5841 - val_accuracy: 0.3206 - val_f1-score: 0.4175
Epoch 48/400
pidor1
601/601 [==============================] - 330s 549ms/step - loss: 0.1285 - accuracy: 0.9450 - f1-score: 0.8795 - val_loss: 0.7581 - val_accuracy: 0.6911 - val_f1-score: 0.2945
Epoch 49/400
pidor1
601/601 [==============================] - 330s 549ms/step - loss: 0.1317 - accuracy: 0.9433 - f1-score: 0.8767 - val_loss: 0.5619 - val_accuracy: 0.3214 - val_f1-score: 0.4380
Epoch 50/400
pidor1
601/601 [======

In [ ]:
res = 'Linknet'
res_l = 'crossentropy'

In [ ]:
    #print(model_str)
    from keras.models import load_model
    size = 256
    with strategy.scope():
        model = models[res](classes = 8, activation = 'softmax')
        #model = load_model('drive/MyDrive/res_weights_Linknet_master_600epochs.tf', compile = False)
        model.compile(optimizer = 'adam', loss = losses['focal_dice_loss'], metrics = ['accuracy', sm.metrics.f1_score])
    print('*'*30)
    print('Fitting model ' + res + f' with {res_l}...')
    print('*'*30)

    train = tf.data.Dataset.from_generator(train_data, output_signature=(
             tf.TensorSpec(shape=(size, size, 3), dtype=tf.float16),
             tf.TensorSpec(shape=(size, size, 8), dtype=tf.float16))).batch(12)

    valid = tf.data.Dataset.from_generator(valid_data, output_signature=(
             tf.TensorSpec(shape=(size, size, 3), dtype=tf.float16),
             tf.TensorSpec(shape=(size, size, 8), dtype=tf.float16))).batch(12)

    epochs = 600
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_f1-score', patience=75)
    model_checkpoint = ModelCheckpoint(f'drive/MyDrive/res_weights_' + res + f'_master_{epochs}epochs_ver2.tf', monitor='val_f1-score', save_best_only=True, mode = 'max')

    history =  model.fit(train, epochs=epochs, verbose=1,
              validation_data = valid,
              callbacks=[model_checkpoint, early_stopping])


******************************
Fitting model Linknet with focal_dice_loss...
******************************
Epoch 1/600
     57/Unknown - 58s 602ms/step - loss: 0.9024 - accuracy: 0.3403 - f1-score: 0.1383

57/57 [==============================] - 81s 1s/step - loss: 0.9024 - accuracy: 0.3403 - f1-score: 0.1383 - val_loss: 1.0503 - val_accuracy: 0.4709 - val_f1-score: 0.0790
Epoch 2/600
57/57 [==============================] - ETA: 0s - loss: 0.8840 - accuracy: 0.3792 - f1-score: 0.1549

57/57 [==============================] - 47s 818ms/step - loss: 0.8840 - accuracy: 0.3792 - f1-score: 0.1549 - val_loss: 1.0265 - val_accuracy: 0.4722 - val_f1-score: 0.0815
Epoch 3/600
57/57 [==============================] - ETA: 0s - loss: 0.8841 - accuracy: 0.3743 - f1-score: 0.1555

57/57 [==============================] - 48s 839ms/step - loss: 0.8841 - accuracy: 0.3743 - f1-score: 0.1555 - val_loss: 0.8984 - val_accuracy: 0.4439 - val_f1-score: 0.1421
Epoch 4/600
57/57 [==============================] - 33s 579ms/step - loss: 0.8805 - accuracy: 0.3924 - f1-score: 0.1589 - val_loss: 0.9439 - val_accuracy: 0.2304 - val_f1-score: 0.1252
Epoch 5/600
57/57 [==============================] - 31s 540ms/step - loss: 0.8796 - accuracy: 0.3674 - f1-score: 0.1615 - val_loss: 0.9328 - val_accuracy: 0.4599 - val_f1-score: 0.1271
Epoch 6/600
57/57 [==============================] - ETA: 0s - loss: 0.8784 - accuracy: 0.3786 - f1-score: 0.1619

57/57 [==============================] - 47s 832ms/step - loss: 0.8784 - accuracy: 0.3786 - f1-score: 0.1619 - val_loss: 0.9063 - val_accuracy: 0.4055 - val_f1-score: 0.1531
Epoch 7/600
57/57 [==============================] - 31s 547ms/step - loss: 0.8799 - accuracy: 0.3620 - f1-score: 0.1618 - val_loss: 0.9100 - val_accuracy: 0.3188 - val_f1-score: 0.1308
Epoch 8/600
57/57 [==============================] - 33s 577ms/step - loss: 0.8787 - accuracy: 0.3725 - f1-score: 0.1617 - val_loss: 0.9055 - val_accuracy: 0.2404 - val_f1-score: 0.1394
Epoch 9/600
57/57 [==============================] - 34s 602ms/step - loss: 0.8783 - accuracy: 0.3753 - f1-score: 0.1618 - val_loss: 1.0886 - val_accuracy: 0.0812 - val_f1-score: 0.0528
Epoch 10/600
57/57 [==============================] - 32s 551ms/step - loss: 0.8779 - accuracy: 0.3702 - f1-score: 0.1639 - val_loss: 0.9457 - val_accuracy: 0.4596 - val_f1-score: 0.1178
Epoch 11/600
57/57 [==============================] - 31s 545ms/step - loss: 0.87

...

57/57 [==============================] - 45s 788ms/step - loss: 0.8483 - accuracy: 0.4071 - f1-score: 0.1972 - val_loss: 0.8671 - val_accuracy: 0.3624 - val_f1-score: 0.1821
Epoch 57/600
57/57 [==============================] - ETA: 0s - loss: 0.8461 - accuracy: 0.4052 - f1-score: 0.1991

57/57 [==============================] - 45s 790ms/step - loss: 0.8461 - accuracy: 0.4052 - f1-score: 0.1991 - val_loss: 0.8693 - val_accuracy: 0.3686 - val_f1-score: 0.1837
Epoch 58/600
57/57 [==============================] - 31s 546ms/step - loss: 0.8437 - accuracy: 0.4139 - f1-score: 0.2017 - val_loss: 0.8745 - val_accuracy: 0.3894 - val_f1-score: 0.1712
Epoch 59/600
57/57 [==============================] - 31s 545ms/step - loss: 0.8451 - accuracy: 0.4094 - f1-score: 0.2021 - val_loss: 0.8780 - val_accuracy: 0.3050 - val_f1-score: 0.1709
Epoch 60/600
57/57 [==============================] - 33s 585ms/step - loss: 0.8431 - accuracy: 0.4140 - f1-score: 0.2031 - val_loss: 0.8758 - val_accuracy: 0.3194 - val_f1-score: 0.1767
Epoch 61/600
57/57 [==============================] - 31s 538ms/step - loss: 0.8432 - accuracy: 0.4077 - f1-score: 0.2024 - val_loss: 0.8722 - val_accuracy: 0.3629 - val_f1-score: 0.1823
Epoch 62/600
57/57 [==============================] - 32s 560ms/step - loss: 0

57/57 [==============================] - 47s 821ms/step - loss: 0.8373 - accuracy: 0.4121 - f1-score: 0.2086 - val_loss: 0.8655 - val_accuracy: 0.3882 - val_f1-score: 0.1874
Epoch 64/600
57/57 [==============================] - 32s 551ms/step - loss: 0.8343 - accuracy: 0.4186 - f1-score: 0.2124 - val_loss: 0.8792 - val_accuracy: 0.3700 - val_f1-score: 0.1704
Epoch 65/600
57/57 [==============================] - 31s 544ms/step - loss: 0.8393 - accuracy: 0.4188 - f1-score: 0.2065 - val_loss: 0.8962 - val_accuracy: 0.3464 - val_f1-score: 0.1573
Epoch 66/600
57/57 [==============================] - 34s 589ms/step - loss: 0.8358 - accuracy: 0.4145 - f1-score: 0.2110 - val_loss: 0.8849 - val_accuracy: 0.3996 - val_f1-score: 0.1695
Epoch 67/600
57/57 [==============================] - 32s 564ms/step - loss: 0.8302 - accuracy: 0.4212 - f1-score: 0.2168 - val_loss: 0.8692 - val_accuracy: 0.3557 - val_f1-score: 0.1810
Epoch 68/600
57/57 [==============================] - 31s 541ms/step - loss: 0

57/57 [==============================] - 45s 787ms/step - loss: 0.8220 - accuracy: 0.4287 - f1-score: 0.2251 - val_loss: 0.8637 - val_accuracy: 0.3904 - val_f1-score: 0.1888
Epoch 71/600
57/57 [==============================] - 33s 568ms/step - loss: 0.8198 - accuracy: 0.4325 - f1-score: 0.2276 - val_loss: 0.8820 - val_accuracy: 0.4120 - val_f1-score: 0.1715
Epoch 72/600
57/57 [==============================] - 33s 582ms/step - loss: 0.8179 - accuracy: 0.4388 - f1-score: 0.2298 - val_loss: 0.8980 - val_accuracy: 0.3581 - val_f1-score: 0.1575
Epoch 73/600
57/57 [==============================] - 31s 539ms/step - loss: 0.8194 - accuracy: 0.4335 - f1-score: 0.2288 - val_loss: 0.8989 - val_accuracy: 0.2775 - val_f1-score: 0.1646
Epoch 74/600
57/57 [==============================] - 31s 544ms/step - loss: 0.8161 - accuracy: 0.4368 - f1-score: 0.2306 - val_loss: 0.9060 - val_accuracy: 0.3493 - val_f1-score: 0.1557
Epoch 75/600
57/57 [==============================] - 31s 541ms/step - loss: 0

57/57 [==============================] - 47s 835ms/step - loss: 0.8121 - accuracy: 0.4437 - f1-score: 0.2361 - val_loss: 0.8644 - val_accuracy: 0.3787 - val_f1-score: 0.1913
Epoch 77/600
57/57 [==============================] - 32s 558ms/step - loss: 0.8044 - accuracy: 0.4498 - f1-score: 0.2443 - val_loss: 0.9198 - val_accuracy: 0.3318 - val_f1-score: 0.1447
Epoch 78/600
57/57 [==============================] - 32s 566ms/step - loss: 0.8040 - accuracy: 0.4510 - f1-score: 0.2449 - val_loss: 0.8803 - val_accuracy: 0.4246 - val_f1-score: 0.1779
Epoch 79/600
57/57 [==============================] - 33s 580ms/step - loss: 0.8079 - accuracy: 0.4445 - f1-score: 0.2404 - val_loss: 0.8661 - val_accuracy: 0.3836 - val_f1-score: 0.1893
Epoch 80/600
57/57 [==============================] - 32s 554ms/step - loss: 0.8014 - accuracy: 0.4540 - f1-score: 0.2471 - val_loss: 0.8761 - val_accuracy: 0.3825 - val_f1-score: 0.1814
Epoch 81/600
57/57 [==============================] - 33s 574ms/step - loss: 0

In [ ]:
res_s = 'images'
"""for key in val_f1:
    if val_f1[res_s] < val_f1[key]:
        res_s = key
print(val_f1)"""

'for key in val_f1:\n    if val_f1[res_s] < val_f1[key]:\n        res_s = key\nprint(val_f1)'

In [ ]:
print(res_s)
model_str = 'Linknet'

images


In [ ]:
s = np.zeros((12))
s1 = 0
for i, j in valid_data('images', 512)():
    for k in range(8):
        s[k] += np.sum(j[:, :, k])
s1 = np.sum(s)
print(s, s1, s / s1)


[772797. 107578. 128232.  17288. 197282.  32748. 369368. 340787.      0.
      0.      0.      0.] 1966080.0 [0.39306488 0.054717   0.06522217 0.00879313 0.10034281 0.01665649
 0.18787028 0.17333323 0.         0.         0.         0.        ]


In [ ]:

print(s[0] / s1)

0.5978628141670322


In [ ]:
w1 = (s1 - s) / s

In [ ]:
print(w1, np.sum(w1))

[ 2.1138221  27.16232807 16.45740858  5.19113138 78.69216389 37.73268569
 69.10751659 70.28039799 15.3036887   4.3197975  19.27120337 15.8816052 ] 361.51374905739374


In [ ]:
w1 /= np.sum(w1)

In [ ]:
w1 = np.array([1/23, 2/23, 2/23, 2/23, 2/23, 2/23, 2/23, 2/23, 2/23, 2/23, 2/23, 2/23])

In [ ]:
import tensorflow_addons as tfa
import sklearn.metrics
with strategy.scope():
    w = np.zeros((12))
    w[1:] = 1 / 11
    f1_score = sm.metrics.FScore()
    print(f1_score)
    dicel = sm.losses.CategoricalCELoss()
    loss = dicel

In [ ]:
    import sklearn
    import tensorflow_addons as tfa
    import keras.backend as K
    from keras import models
    from keras.layers import Input, Conv2D, MaxPooling2D, Dropout, concatenate, UpSampling2D
    def dice_coef(smooth=1e-6, w = 1):
        def dice(y_true, y_pred):
            y_true_f = K.flatten(y_true)
            y_pred_f = K.flatten(y_pred)
            #print(y_true.shape, y_pred.shape)
            #print((y_true_f * y_pred_f).shape)
            intersection = K.sum(y_true_f * y_pred_f)
            return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
        return dice

#def bincross()

    def dice_coef_loss(y_true, y_pred):
        return 1-dice_coef(w = 1)(y_true, y_pred)

    def iou_coef(smooth=1e-6):
        def iou(y_true, y_pred):
            intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
            union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
            iou1 = K.mean((intersection + smooth) / (union + smooth), axis=0)
            return iou1
        return iou


    def unet(input_size = (None, None, 3), nc = 3):
        inputs = keras.layers.Input(input_size)
        dr =  MaxPooling2D(pool_size=(4, 4))(inputs)

        conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same',
                                                   kernel_initializer = 'he_normal'
                                                   )(dr)

        conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same',
                                                   kernel_initializer = 'he_normal'
                                                   )(conv1)

        pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

        conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same',
                                                    kernel_initializer = 'he_normal'
                                                    )(pool1)

        conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same',
                                                    kernel_initializer = 'he_normal'
                                                    )(conv2)

        pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

        conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same',
                                                    kernel_initializer = 'he_normal'
                                                    )(pool2)

        conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same',
                                                    kernel_initializer = 'he_normal'
                                                    )(conv3)

        pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

        conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same',
                                                    kernel_initializer = 'he_normal'
                                                    )(pool3)

        conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same',
                                                    kernel_initializer = 'he_normal'
                                                    )(conv4)

        drop4 = Dropout(0.5)(conv4)

        pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

        conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same',
                                                     kernel_initializer = 'he_normal'
                                                     )(pool4)

        conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same',
                                                     kernel_initializer = 'he_normal'
                                                     )(conv5)

        drop5 = Dropout(0.5)(conv5)

        up6 = Conv2D(512, 2, activation = 'relu', padding = 'same',
                                                  kernel_initializer = 'he_normal'
                                                  )(UpSampling2D(size = (2,2))(drop5))

        merge6 = concatenate([drop4,up6])

        conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same',
                                                    kernel_initializer = 'he_normal'
                                                    )(merge6)

        conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same',
                                                    kernel_initializer = 'he_normal'
                                                    )(conv6)

        up7 = Conv2D(256, 2, activation = 'relu', padding = 'same',
                                                  kernel_initializer = 'he_normal'
                                                  )(UpSampling2D(size = (2,2))(conv6))

        merge7 = concatenate([conv3,up7])

        conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same',
                                                    kernel_initializer = 'he_normal'
                                                    )(merge7)

        conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal'
                                                    )(conv7)


        up8 = Conv2D(128, 2, activation = 'relu', padding = 'same',
                                                  kernel_initializer = 'he_normal'
                                                  )(UpSampling2D(size = (2,2))(conv7))

        merge8 = concatenate([conv2,up8])

        conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same',
                                                    kernel_initializer = 'he_normal'
                                                    )(merge8)

        conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same',
                                                    kernel_initializer = 'he_normal'
                                                    )(conv8)

        up9 = Conv2D(64, 2, activation = 'relu', padding = 'same',
                                                 kernel_initializer = 'he_normal'
                                                 )(UpSampling2D(size = (2,2))(conv8))

        merge9 = concatenate([conv1,up9])

        conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same',
                                                   kernel_initializer = 'he_normal'
                                                   )(merge9)

        conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same',
                                                   kernel_initializer = 'he_normal'
                                                   )(conv9)

        conv9 = Conv2D(nc, 3, activation = 'sigmoid', padding = 'same',
                                                 kernel_initializer = 'he_normal'
                                                  )(UpSampling2D(size = (4, 4))(conv9))

        #conv10 = Conv2D(12, 1, activation = 'sigmoid')(UpSampling2D(size = (4, 4))(conv9))

        model = keras.Model(inputs = inputs, outputs = conv9)
        #w= [0.005, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
        model.compile(optimizer = keras.optimizers.Adam(learning_rate = 3e-5), loss = dice_coef_loss,
                      metrics = [dice_coef(), iou_coef(), 'accuracy'])
        return model
    epochs = 400
    s = 'images'
    size = 512
    train = tf.data.Dataset.from_generator(train_data1(s, size), output_signature=(
             tf.TensorSpec(shape=(size, size, 3), dtype=tf.float16),
             tf.TensorSpec(shape=(size, size, 1), dtype=tf.float16))).batch(6)

    valid = tf.data.Dataset.from_generator(valid_data1(s, size), output_signature=(
             tf.TensorSpec(shape=(size, size, 3), dtype=tf.float16),
             tf.TensorSpec(shape=(size, size, 1), dtype=tf.float16))).batch(6)
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_mse', patience=100)
    model_checkpoint = ModelCheckpoint(f'weights_new_clouds_removal_master_{epochs}epochs.tf', monitor='val_accuracy', save_best_only=True, mode = 'max')
    model = unet(nc = 1)
    #model.compile(optimizer = keras.optimizers.Adam(learning_rate = 3e-5), loss = dice_coef_loss, metrics = [dice_coef(w = 1), iou_coef(), 'accuracy'])
    history =  model.fit(train, epochs=epochs, verbose=1,
              validation_data = valid,
              callbacks=[model_checkpoint, early_stopping])


Epoch 1/400
pidor1
   1129/Unknown - 549s 485ms/step - loss: 0.3863 - dice: 0.6135 - iou: 0.3216 - accuracy: 0.6764pidor1


C:\Users\CLD010\AppData\Local\Temp\ipykernel_15096\218061911.py:51: RuntimeWarning: invalid value encountered in divide
  img_train = np.asarray(img).astype('float16') / np.max(img)
C:\Users\CLD010\AppData\Local\Temp\ipykernel_15096\218061911.py:58: RuntimeWarning: invalid value encountered in divide
  mask_train = np.asarray(img).astype('float16').reshape((512, 512, 1)) / np.max(img)


INFO:tensorflow:Assets written to: weights_new_clouds_removal_master_400epochs.tf\assets


INFO:tensorflow:Assets written to: weights_new_clouds_removal_master_400epochs.tf\assets


1129/1129 [==============================] - 705s 623ms/step - loss: 0.3863 - dice: 0.6135 - iou: 0.3216 - accuracy: 0.6764 - val_loss: nan - val_dice: nan - val_iou: nan - val_accuracy: 0.3670
Epoch 2/400
pidor1
1129/1129 [==============================] - ETA: 0s - loss: 0.3742 - dice: 0.6259 - iou: 0.3343 - accuracy: 0.6863pidor1


...

INFO:tensorflow:Assets written to: weights_new_clouds_removal_master_400epochs.tf\assets


INFO:tensorflow:Assets written to: weights_new_clouds_removal_master_400epochs.tf\assets


1129/1129 [==============================] - 455s 403ms/step - loss: 0.2230 - dice: 0.7769 - iou: 0.3970 - accuracy: 0.8746 - val_loss: nan - val_dice: nan - val_iou: nan - val_accuracy: 0.4941
Epoch 8/400
pidor1
1129/1129 [==============================] - ETA: 0s - loss: 0.2009 - dice: 0.7990 - iou: 0.3985 - accuracy: 0.8913pidor1


C:\Users\CLD010\AppData\Local\Temp\ipykernel_15096\218061911.py:58: RuntimeWarning: invalid value encountered in divide
  mask_train = np.asarray(img).astype('float16').reshape((512, 512, 1)) / np.max(img)
C:\Users\CLD010\AppData\Local\Temp\ipykernel_15096\218061911.py:51: RuntimeWarning: invalid value encountered in divide
  img_train = np.asarray(img).astype('float16') / np.max(img)


1129/1129 [==============================] - 446s 395ms/step - loss: 0.2009 - dice: 0.7990 - iou: 0.3985 - accuracy: 0.8913 - val_loss: nan - val_dice: nan - val_iou: nan - val_accuracy: 0.4769
Epoch 9/400
pidor1
1022/1129 [==========================>...] - ETA: 34s - loss: 0.1903 - dice: 0.8097 - iou: 0.4054 - accuracy: 0.8987

In [ ]:
path = 'D:/Misha/master/images/divided'
l = os.listdir(os.path.join(path, 'test'))
for i in l:
    img = np.array(Image.open(os.path.join(path, 'test', i))) / 255
    r1 = model.predict(np.array([img]))[0]
    print(np.max(img), np.max(r1))
    print(np.mean((img - r1) ** 2))
    x = cv2.imwrite(os.path.join(path, 'no_clouds', i), r1 * 255)
    if not x:
        print(i)

1/1 [==============================] - 0s 260ms/step
1.0 0.84099364
0.008453559326449287
1/1 [==============================] - 0s 30ms/step
1.0 0.9902811
0.016892526693716315
1/1 [==============================] - 0s 29ms/step
1.0 0.866807
0.011871809156028404
1/1 [==============================] - 0s 29ms/step
1.0 0.7322711
0.01629989406603383
1/1 [==============================] - 0s 29ms/step
0.09411764705882353 0.4191911
0.008806284703321872
1/1 [==============================] - 0s 30ms/step
1.0 0.9880121
0.014409617794263316
1/1 [==============================] - 0s 30ms/step
1.0 0.98743665
0.014438251939887526
1/1 [==============================] - 0s 30ms/step
1.0 0.98885894
0.012856983108816231
1/1 [==============================] - 0s 29ms/step
0.7529411764705882 0.57829875
0.010896188273779398
1/1 [==============================] - 0s 29ms/step
1.0 0.99102825
0.013066824870836756
1/1 [==============================] - 0s 30ms/step
1.0 0.91978556
0.009592205375784275
1/1 [==

1/1 [==============================] - 0s 30ms/step
1.0 0.97954226
0.00949791129379337
1/1 [==============================] - 0s 30ms/step
1.0 0.925567
0.009919282567340227
1/1 [==============================] - 0s 29ms/step
1.0 0.9772529
0.004234079935498036
1/1 [==============================] - 0s 29ms/step
0.4470588235294118 0.49201328
0.013968077757178692
1/1 [==============================] - 0s 30ms/step
0.48627450980392156 0.49000144
0.005671893089736586
1/1 [==============================] - 0s 30ms/step
1.0 0.88253266
0.0035422833604795683
1/1 [==============================] - 0s 30ms/step
1.0 0.9347977
0.013174494886785878
1/1 [==============================] - 0s 30ms/step
1.0 0.89185536
0.007758796447688857
1/1 [==============================] - 0s 30ms/step
1.0 0.90608126
0.006361694018401848
1/1 [==============================] - 0s 30ms/step
0.1803921568627451 0.3922222
0.002604241561312048
1/1 [==============================] - 0s 30ms/step
1.0 0.91141075
0.0043154694

In [ ]:
s = 0
s1 = 0
for i, j in train_data():
  s += np.sum(j[:, :, 0])
  s1 += j.shape[0] * j.shape[1]
print(s, s1)

49960583.0 94765056


In [ ]:
print(s / s1)

0.5272047008551338


In [ ]:
    import sklearn
    import keras.backend as K
    from keras import models
    from keras.layers import Input, Conv2D, MaxPooling2D, Dropout, concatenate, UpSampling2D
    def dice_coef(smooth=1e-6, w = 1):
        def dice(y_true, y_pred):
            y_true_f = K.flatten(y_true)
            y_pred_f = K.flatten(y_pred)
            #print((y_true_f * y_pred_f).shape)
            intersection = K.sum(y_true_f * y_pred_f)
            return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
        return dice

#def bincross()

    def dice_coef_loss(y_true, y_pred):
        return 1-dice_coef(w = 1)(y_true, y_pred)

    def iou_coef(smooth=1e-6):
        def iou(y_true, y_pred):
            intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
            union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
            iou1 = K.mean((intersection + smooth) / (union + smooth), axis=0)
            return iou1
        return iou


    def unet(input_size = (None, None, 3), nc = 9):
        inputs = keras.layers.Input(input_size)
        dr =  MaxPooling2D(pool_size=(4, 4))(inputs)

        conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same',
                                                   kernel_initializer = 'he_normal'
                                                   )(dr)

        conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same',
                                                   kernel_initializer = 'he_normal'
                                                   )(conv1)

        pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

        conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same',
                                                    kernel_initializer = 'he_normal'
                                                    )(pool1)

        conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same',
                                                    kernel_initializer = 'he_normal'
                                                    )(conv2)

        pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

        conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same',
                                                    kernel_initializer = 'he_normal'
                                                    )(pool2)

        conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same',
                                                    kernel_initializer = 'he_normal'
                                                    )(conv3)

        pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

        conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same',
                                                    kernel_initializer = 'he_normal'
                                                    )(pool3)

        conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same',
                                                    kernel_initializer = 'he_normal'
                                                    )(conv4)

        drop4 = Dropout(0.5)(conv4)

        pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

        conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same',
                                                     kernel_initializer = 'he_normal'
                                                     )(pool4)

        conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same',
                                                     kernel_initializer = 'he_normal'
                                                     )(conv5)

        drop5 = Dropout(0.5)(conv5)

        up6 = Conv2D(512, 2, activation = 'relu', padding = 'same',
                                                  kernel_initializer = 'he_normal'
                                                  )(UpSampling2D(size = (2,2))(drop5))

        merge6 = concatenate([drop4,up6])

        conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same',
                                                    kernel_initializer = 'he_normal'
                                                    )(merge6)

        conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same',
                                                    kernel_initializer = 'he_normal'
                                                    )(conv6)

        up7 = Conv2D(256, 2, activation = 'relu', padding = 'same',
                                                  kernel_initializer = 'he_normal'
                                                  )(UpSampling2D(size = (2,2))(conv6))

        merge7 = concatenate([conv3,up7])

        conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same',
                                                    kernel_initializer = 'he_normal'
                                                    )(merge7)

        conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal'
                                                    )(conv7)


        up8 = Conv2D(128, 2, activation = 'relu', padding = 'same',
                                                  kernel_initializer = 'he_normal'
                                                  )(UpSampling2D(size = (2,2))(conv7))

        merge8 = concatenate([conv2,up8])

        conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same',
                                                    kernel_initializer = 'he_normal'
                                                    )(merge8)

        conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same',
                                                    kernel_initializer = 'he_normal'
                                                    )(conv8)

        up9 = Conv2D(64, 2, activation = 'relu', padding = 'same',
                                                 kernel_initializer = 'he_normal'
                                                 )(UpSampling2D(size = (2,2))(conv8))

        merge9 = concatenate([conv1,up9])

        conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same',
                                                   kernel_initializer = 'he_normal'
                                                   )(merge9)

        conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same',
                                                  kernel_initializer = 'he_normal'
                                                  )(conv9)

        conv9 = Conv2D(32, 3, activation = 'relu', padding = 'same',
                                                 kernel_initializer = 'he_normal'
                                                  )(conv9)

        conv10 = Conv2D(nc, 1, activation = 'softmax')(UpSampling2D(size = (4, 4))(conv9))

        model = keras.Model(inputs = inputs, outputs = conv10)
        #w= [0.005, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
        model.compile(optimizer = keras.optimizers.Adam(learning_rate = 3e-5), loss = losses['focal_dice_loss'],
                      metrics = ['accuracy', sm.metrics.f1_score])
        return model

    epochs = 800
    size = 256
    train = tf.data.Dataset.from_generator(train_data, output_signature=(
             tf.TensorSpec(shape=(size, size, 3), dtype=tf.float16),
             tf.TensorSpec(shape=(size, size, 8), dtype=tf.float16))).batch(12)

    valid = tf.data.Dataset.from_generator(valid_data, output_signature=(
             tf.TensorSpec(shape=(size, size, 3), dtype=tf.float16),
             tf.TensorSpec(shape=(size, size, 8), dtype=tf.float16))).batch(12)
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_f1-score', patience=100)
    model_checkpoint = ModelCheckpoint(f'content/drive/MyDrive/weights_with_clouds_1_master_{epochs}epochs_ver2.tf',
                                       monitor='val_f1-score', save_best_only=True, mode = 'max')
    model = unet(nc = 8)
    #model.compile(optimizer = keras.optimizers.Adam(learning_rate = 3e-5), loss = dice_coef_loss, metrics = [dice_coef(w = 1), iou_coef(), 'accuracy'])
    history =  model.fit(train, epochs=epochs, verbose=1,
              validation_data = valid,
              callbacks=[model_checkpoint, early_stopping])


Epoch 1/800
     57/Unknown - 61s 467ms/step - loss: 0.9252 - accuracy: 0.4118 - f1-score: 0.1146

57/57 [==============================] - 72s 667ms/step - loss: 0.9252 - accuracy: 0.4118 - f1-score: 0.1146 - val_loss: 0.9167 - val_accuracy: 0.4633 - val_f1-score: 0.1207
Epoch 2/800
57/57 [==============================] - ETA: 0s - loss: 0.9118 - accuracy: 0.4843 - f1-score: 0.1242

57/57 [==============================] - 34s 596ms/step - loss: 0.9118 - accuracy: 0.4843 - f1-score: 0.1242 - val_loss: 0.9117 - val_accuracy: 0.4699 - val_f1-score: 0.1252
Epoch 3/800
57/57 [==============================] - 28s 485ms/step - loss: 0.9038 - accuracy: 0.4844 - f1-score: 0.1321 - val_loss: 0.9142 - val_accuracy: 0.3759 - val_f1-score: 0.1233
Epoch 4/800
57/57 [==============================] - ETA: 0s - loss: 0.8990 - accuracy: 0.4559 - f1-score: 0.1381

57/57 [==============================] - 34s 595ms/step - loss: 0.8990 - accuracy: 0.4559 - f1-score: 0.1381 - val_loss: 0.9090 - val_accuracy: 0.4295 - val_f1-score: 0.1285
Epoch 5/800
57/57 [==============================] - ETA: 0s - loss: 0.8940 - accuracy: 0.4430 - f1-score: 0.1432

57/57 [==============================] - 33s 583ms/step - loss: 0.8940 - accuracy: 0.4430 - f1-score: 0.1432 - val_loss: 0.9049 - val_accuracy: 0.3895 - val_f1-score: 0.1327
Epoch 6/800
57/57 [==============================] - ETA: 0s - loss: 0.8907 - accuracy: 0.4241 - f1-score: 0.1472

...

57/57 [==============================] - 34s 591ms/step - loss: 0.6891 - accuracy: 0.6113 - f1-score: 0.3576 - val_loss: 0.8711 - val_accuracy: 0.3863 - val_f1-score: 0.2025
Epoch 76/800
57/57 [==============================] - 28s 494ms/step - loss: 0.6936 - accuracy: 0.6100 - f1-score: 0.3538 - val_loss: 0.8842 - val_accuracy: 0.3585 - val_f1-score: 0.1861
Epoch 77/800
57/57 [==============================] - 28s 484ms/step - loss: 0.6901 - accuracy: 0.6131 - f1-score: 0.3560 - val_loss: 0.8802 - val_accuracy: 0.3589 - val_f1-score: 0.1945
Epoch 78/800
57/57 [==============================] - 27s 480ms/step - loss: 0.6844 - accuracy: 0.6125 - f1-score: 0.3627 - val_loss: 0.8868 - val_accuracy: 0.3603 - val_f1-score: 0.1848
Epoch 79/800
57/57 [==============================] - 27s 478ms/step - loss: 0.6847 - accuracy: 0.6143 - f1-score: 0.3621 - val_loss: 0.8699 - val_accuracy: 0.3604 - val_f1-score: 0.2016
Epoch 80/800
57/57 [==============================] - 27s 476ms/step - loss: 0

57/57 [==============================] - 34s 597ms/step - loss: 0.6773 - accuracy: 0.6195 - f1-score: 0.3690 - val_loss: 0.8699 - val_accuracy: 0.3709 - val_f1-score: 0.2071
Epoch 84/800
57/57 [==============================] - 29s 505ms/step - loss: 0.6703 - accuracy: 0.6216 - f1-score: 0.3771 - val_loss: 0.8895 - val_accuracy: 0.3451 - val_f1-score: 0.1896
Epoch 85/800
57/57 [==============================] - ETA: 0s - loss: 0.6728 - accuracy: 0.6222 - f1-score: 0.3735

57/57 [==============================] - 34s 596ms/step - loss: 0.6728 - accuracy: 0.6222 - f1-score: 0.3735 - val_loss: 0.8643 - val_accuracy: 0.3872 - val_f1-score: 0.2105
Epoch 86/800
57/57 [==============================] - 27s 471ms/step - loss: 0.6725 - accuracy: 0.6241 - f1-score: 0.3741 - val_loss: 0.8804 - val_accuracy: 0.3776 - val_f1-score: 0.1950
Epoch 87/800
57/57 [==============================] - ETA: 0s - loss: 0.6707 - accuracy: 0.6252 - f1-score: 0.3766

57/57 [==============================] - 34s 592ms/step - loss: 0.6707 - accuracy: 0.6252 - f1-score: 0.3766 - val_loss: 0.8569 - val_accuracy: 0.4037 - val_f1-score: 0.2116
Epoch 88/800
57/57 [==============================] - 30s 515ms/step - loss: 0.6708 - accuracy: 0.6250 - f1-score: 0.3760 - val_loss: 0.8782 - val_accuracy: 0.3470 - val_f1-score: 0.2044
Epoch 89/800
57/57 [==============================] - 28s 493ms/step - loss: 0.6700 - accuracy: 0.6258 - f1-score: 0.3786 - val_loss: 0.8689 - val_accuracy: 0.3879 - val_f1-score: 0.2036
Epoch 90/800
57/57 [==============================] - 28s 484ms/step - loss: 0.6711 - accuracy: 0.6237 - f1-score: 0.3769 - val_loss: 0.8783 - val_accuracy: 0.3547 - val_f1-score: 0.2047
Epoch 91/800
57/57 [==============================] - 26s 453ms/step - loss: 0.6610 - accuracy: 0.6285 - f1-score: 0.3852 - val_loss: 0.8785 - val_accuracy: 0.3667 - val_f1-score: 0.1974
Epoch 92/800
57/57 [==============================] - 26s 460ms/step - loss: 0

57/57 [==============================] - 33s 574ms/step - loss: 0.6533 - accuracy: 0.6350 - f1-score: 0.3926 - val_loss: 0.8611 - val_accuracy: 0.4047 - val_f1-score: 0.2120
Epoch 98/800
57/57 [==============================] - 27s 476ms/step - loss: 0.6555 - accuracy: 0.6339 - f1-score: 0.3896 - val_loss: 0.8689 - val_accuracy: 0.3731 - val_f1-score: 0.2025
Epoch 99/800
57/57 [==============================] - 27s 472ms/step - loss: 0.6518 - accuracy: 0.6359 - f1-score: 0.3949 - val_loss: 0.8684 - val_accuracy: 0.3921 - val_f1-score: 0.2066
Epoch 100/800
57/57 [==============================] - 27s 469ms/step - loss: 0.6493 - accuracy: 0.6374 - f1-score: 0.3987 - val_loss: 0.8732 - val_accuracy: 0.3919 - val_f1-score: 0.2040
Epoch 101/800
57/57 [==============================] - 26s 464ms/step - loss: 0.6483 - accuracy: 0.6384 - f1-score: 0.3979 - val_loss: 0.8751 - val_accuracy: 0.3732 - val_f1-score: 0.2041


In [ ]:
for i in train_data('images', 256)():
    print(np.max(i[1][:, 8]))

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0


KeyboardInterrupt: 

In [ ]:
    #СЕЙЧАС ОБУЧАЮ!!!
    #СЕЙЧАС ОБУЧАЮ!!!
    #СЕЙЧАС ОБУЧАЮ!!!
    import tensorflow_addons as tfa
    import sklearn.metrics
    from keras import Model
    models = {'Linknet':sm.Linknet, 'FPN':sm.FPN, 'Unet': sm.Unet}
    res = 'Linknet'
    with strategy.scope():
        w = np.zeros((12))
        w[1:] = 1 / 11
        f1_score = sm.metrics.FScore()
        print(f1_score)
        dicel = sm.losses.DiceLoss()
        loss = dicel
    def af(y_t, y_p):
        return f1_score(y_t, y_p) * tf.keras.metrics.Accuracy()(y_t, y_p)
    import sklearn
    import tensorflow_addons as tfa
    res_l = 'focal_dice_loss'
    losses = {'jaccard_loss' : sm.losses.jaccard_loss, 'dice_loss' : sm.losses.dice_loss, 'focal_loss' : sm.losses.categorical_focal_loss, 'crossentropy' : sm.losses.categorical_crossentropy, 'cce_dice_loss' : sm.losses.cce_dice_loss, 'bce_jaccard_loss' : sm.losses.cce_jaccard_loss, 'focal_dice_loss' : sm.losses.categorical_focal_dice_loss, 'focal_jaccard_loss' : sm.losses.categorical_focal_jaccard_loss}
    size = 512
    s = 'sr'
    with strategy.scope():
        #f1_score = tfa.metrics.F1Score(num_classes=13, threshold=0.5, average = 'weighted')
        model = models[res](classes = 8, activation = 'softmax', encoder_freeze = True)
        model.compile(optimizer = 'rmsprop', loss = losses[res_l], metrics = ['accuracy', f1_score])
    print('*'*30)
    print('Fitting model ' + res + '...')
    print('*'*30)
    s = 'images'
    size = 256
    train = tf.data.Dataset.from_generator(train_data(s, size = size), output_signature=(
             tf.TensorSpec(shape=(size, size, 3), dtype=tf.float16),
             tf.TensorSpec(shape=(size, size, 8), dtype=tf.float16))).batch(6)

    valid = tf.data.Dataset.from_generator(valid_data(s, size = size), output_signature=(
             tf.TensorSpec(shape=(size, size, 3), dtype=tf.float16),
             tf.TensorSpec(shape=(size, size, 8), dtype=tf.float16))).batch(6)

    epochs = 800
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=100)
    model_checkpoint = ModelCheckpoint(f'weights_final3_master_{epochs}epochs_new.tf', monitor='val_accuracy', save_best_only=True, mode = 'max')
    #model = keras.models.load_model('weights_final3_master_400epochs_new.tf', compile = False)
    #model.compile(optimizer = 'rmsprop', loss = losses[res_l], metrics = ['accuracy', f1_score])
    history =  model.fit(train, epochs=epochs, verbose=1,
              validation_data = valid,
              callbacks=[model_checkpoint, early_stopping])


******************************
Fitting model Linknet...
******************************
Epoch 1/800
    120/Unknown - 38s 212ms/step - loss: 0.9082 - accuracy: 0.4036 - f1-score: 0.1331

INFO:tensorflow:Assets written to: weights_final3_master_800epochs_new.tf\assets


INFO:tensorflow:Assets written to: weights_final3_master_800epochs_new.tf\assets


120/120 [==============================] - 54s 342ms/step - loss: 0.9082 - accuracy: 0.4036 - f1-score: 0.1331 - val_loss: 0.9263 - val_accuracy: 0.3684 - val_f1-score: 0.1270
Epoch 2/800
120/120 [==============================] - 26s 214ms/step - loss: 0.8981 - accuracy: 0.3987 - f1-score: 0.1433 - val_loss: 0.9373 - val_accuracy: 0.2479 - val_f1-score: 0.1262
Epoch 3/800
120/120 [==============================] - 26s 212ms/step - loss: 0.8924 - accuracy: 0.4052 - f1-score: 0.1489 - val_loss: 0.9550 - val_accuracy: 0.3348 - val_f1-score: 0.1336
Epoch 4/800
120/120 [==============================] - 26s 214ms/step - loss: 0.8865 - accuracy: 0.4161 - f1-score: 0.1558 - val_loss: 0.9102 - val_accuracy: 0.2960 - val_f1-score: 0.1442
Epoch 5/800
120/120 [==============================] - 25s 211ms/step - loss: 0.8847 - accuracy: 0.4139 - f1-score: 0.1578 - val_loss: 0.9827 - val_accuracy: 0.3557 - val_f1-score: 0.1290
Epoch 6/800
120/120 [==============================] - ETA: 0s - loss: 0

INFO:tensorflow:Assets written to: weights_final3_master_800epochs_new.tf\assets


...

INFO:tensorflow:Assets written to: weights_final3_master_800epochs_new.tf\assets


INFO:tensorflow:Assets written to: weights_final3_master_800epochs_new.tf\assets


120/120 [==============================] - 36s 299ms/step - loss: 0.3278 - accuracy: 0.8844 - f1-score: 0.6930 - val_loss: 0.8198 - val_accuracy: 0.5000 - val_f1-score: 0.2815
Epoch 380/800
120/120 [==============================] - 25s 210ms/step - loss: 0.3217 - accuracy: 0.8847 - f1-score: 0.6987 - val_loss: 0.8337 - val_accuracy: 0.4831 - val_f1-score: 0.2666
Epoch 381/800
120/120 [==============================] - 25s 209ms/step - loss: 0.3246 - accuracy: 0.8835 - f1-score: 0.6962 - val_loss: 0.8330 - val_accuracy: 0.4826 - val_f1-score: 0.2669
Epoch 382/800
120/120 [==============================] - 25s 209ms/step - loss: 0.3201 - accuracy: 0.8843 - f1-score: 0.7007 - val_loss: 0.8200 - val_accuracy: 0.4887 - val_f1-score: 0.2852
Epoch 383/800
120/120 [==============================] - 25s 207ms/step - loss: 0.3187 - accuracy: 0.8846 - f1-score: 0.7020 - val_loss: 0.8167 - val_accuracy: 0.4863 - val_f1-score: 0.2783
Epoch 384/800
120/120 [==============================] - 25s 208

120/120 [==============================] - 25s 209ms/step - loss: 0.3214 - accuracy: 0.8875 - f1-score: 0.6987 - val_loss: 0.8398 - val_accuracy: 0.4968 - val_f1-score: 0.2587
Epoch 423/800
120/120 [==============================] - 25s 204ms/step - loss: 0.3167 - accuracy: 0.8874 - f1-score: 0.7037 - val_loss: 0.8264 - val_accuracy: 0.4836 - val_f1-score: 0.2710
Epoch 424/800
120/120 [==============================] - 26s 212ms/step - loss: 0.3219 - accuracy: 0.8866 - f1-score: 0.6987 - val_loss: 0.8274 - val_accuracy: 0.4740 - val_f1-score: 0.2663
Epoch 425/800
120/120 [==============================] - 25s 209ms/step - loss: 0.3141 - accuracy: 0.8877 - f1-score: 0.7058 - val_loss: 0.8308 - val_accuracy: 0.4786 - val_f1-score: 0.2655
Epoch 426/800
120/120 [==============================] - 25s 207ms/step - loss: 0.3146 - accuracy: 0.8878 - f1-score: 0.7053 - val_loss: 0.8515 - val_accuracy: 0.4869 - val_f1-score: 0.2450
Epoch 427/800
120/120 [==============================] - 25s 209

120/120 [==============================] - 26s 213ms/step - loss: 0.3107 - accuracy: 0.8900 - f1-score: 0.7089 - val_loss: 0.8290 - val_accuracy: 0.4873 - val_f1-score: 0.2810
Epoch 466/800
120/120 [==============================] - 26s 213ms/step - loss: 0.3137 - accuracy: 0.8895 - f1-score: 0.7055 - val_loss: 0.8436 - val_accuracy: 0.4914 - val_f1-score: 0.2556
Epoch 467/800
120/120 [==============================] - 25s 209ms/step - loss: 0.3099 - accuracy: 0.8900 - f1-score: 0.7096 - val_loss: 0.8321 - val_accuracy: 0.4881 - val_f1-score: 0.2678
Epoch 468/800
120/120 [==============================] - 26s 212ms/step - loss: 0.3099 - accuracy: 0.8900 - f1-score: 0.7095 - val_loss: 0.8424 - val_accuracy: 0.4928 - val_f1-score: 0.2655
Epoch 469/800
120/120 [==============================] - 26s 216ms/step - loss: 0.3097 - accuracy: 0.8901 - f1-score: 0.7098 - val_loss: 0.8168 - val_accuracy: 0.4871 - val_f1-score: 0.2798
Epoch 470/800
120/120 [==============================] - 25s 211

In [ ]:
model.save('weights_final3_master_400epochs.tf')

INFO:tensorflow:Assets written to: weights_final3_master_400epochs.tf\assets


INFO:tensorflow:Assets written to: weights_final3_master_400epochs.tf\assets


In [ ]:
    import tensorflow_addons as tfa
    import sklearn.metrics
    with strategy.scope():
        w = np.zeros((12))
        w[1:] = 1 / 11
        f1_score = sm.metrics.FScore()
        print(f1_score)
        dicel = sm.losses.DiceLoss()
        loss = dicel
    import sklearn
    import tensorflow_addons as tfa
    model_str = 'Linknet'
    losses = {'jaccard_loss' : sm.losses.jaccard_loss, 'dice_loss' : sm.losses.dice_loss, 'focal_loss' : sm.losses.categorical_focal_loss, 'crossentropy' : sm.losses.categorical_crossentropy, 'cce_dice_loss' : sm.losses.cce_dice_loss, 'bce_jaccard_loss' : sm.losses.cce_jaccard_loss, 'focal_dice_loss' : sm.losses.categorical_focal_dice_loss, 'focal_jaccard_loss' : sm.losses.categorical_focal_jaccard_loss}
    size = 256
    s = 'images'
    with strategy.scope():
        #f1_score = tfa.metrics.F1Score(num_classes=13, threshold=0.5, average = 'weighted')
        model = sm.Linknet(classes = 9, activation = 'softmax')
        model.compile(optimizer = 'Adam', loss = sm.losses.dice_loss, metrics = ['accuracy', f1_score])
    print('*'*30)
    print('Fitting model ' + model_str + '...')
    print('*'*30)

    train = tf.data.Dataset.from_generator(train_data(s, size = size), output_signature=(
             tf.TensorSpec(shape=(size, size, 3), dtype=tf.float16),
             tf.TensorSpec(shape=(size, size, 9), dtype=tf.float16))).batch(6)

    valid = tf.data.Dataset.from_generator(valid_data(s, size = size), output_signature=(
             tf.TensorSpec(shape=(size, size, 3), dtype=tf.float16),
             tf.TensorSpec(shape=(size, size, 9), dtype=tf.float16))).batch(6)

    epochs = 400
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=45)
    model_checkpoint = ModelCheckpoint(f'weights_final4_master_{epochs}epochs.tf', monitor='val_accuracy', save_best_only=True, mode = 'max')

    history =  model.fit(train, epochs=epochs, verbose=1,
              validation_data = valid,
              callbacks=[model_checkpoint, early_stopping])


******************************
Fitting model Linknet...
******************************
Epoch 1/400
     74/Unknown - 39s 337ms/step - loss: 0.8653 - accuracy: 0.2341 - f1-score: 0.1347

INFO:tensorflow:Assets written to: weights_final4_master_400epochs.tf\assets


INFO:tensorflow:Assets written to: weights_final4_master_400epochs.tf\assets


74/74 [==============================] - 58s 589ms/step - loss: 0.8653 - accuracy: 0.2341 - f1-score: 0.1347 - val_loss: 0.9266 - val_accuracy: 0.1797 - val_f1-score: 0.0734
Epoch 2/400
74/74 [==============================] - ETA: 0s - loss: 0.8478 - accuracy: 0.2444 - f1-score: 0.1522

INFO:tensorflow:Assets written to: weights_final4_master_400epochs.tf\assets


...

74/74 [==============================] - 40s 539ms/step - loss: 0.8325 - accuracy: 0.2508 - f1-score: 0.1675 - val_loss: 0.8682 - val_accuracy: 0.4967 - val_f1-score: 0.1318
Epoch 9/400
74/74 [==============================] - 29s 391ms/step - loss: 0.8296 - accuracy: 0.2554 - f1-score: 0.1704 - val_loss: 0.9149 - val_accuracy: 0.1184 - val_f1-score: 0.0851
Epoch 10/400
74/74 [==============================] - 29s 389ms/step - loss: 0.8297 - accuracy: 0.2550 - f1-score: 0.1703 - val_loss: 0.9525 - val_accuracy: 0.0606 - val_f1-score: 0.0475
Epoch 11/400
74/74 [==============================] - 29s 389ms/step - loss: 0.8248 - accuracy: 0.2567 - f1-score: 0.1752 - val_loss: 0.9099 - val_accuracy: 0.1215 - val_f1-score: 0.0901
Epoch 12/400
74/74 [==============================] - 29s 385ms/step - loss: 0.8255 - accuracy: 0.2571 - f1-score: 0.1745 - val_loss: 0.8653 - val_accuracy: 0.3198 - val_f1-score: 0.1347
Epoch 13/400
74/74 [==============================] - 29s 387ms/step - loss: 0.

INFO:tensorflow:Assets written to: weights_final4_master_400epochs.tf\assets


INFO:tensorflow:Assets written to: weights_final4_master_400epochs.tf\assets


74/74 [==============================] - 41s 551ms/step - loss: 0.8182 - accuracy: 0.2686 - f1-score: 0.1818 - val_loss: 0.8655 - val_accuracy: 0.5246 - val_f1-score: 0.1345
Epoch 28/400
74/74 [==============================] - 29s 385ms/step - loss: 0.8209 - accuracy: 0.2621 - f1-score: 0.1791 - val_loss: 0.8712 - val_accuracy: 0.2994 - val_f1-score: 0.1288
Epoch 29/400
74/74 [==============================] - 28s 383ms/step - loss: 0.8135 - accuracy: 0.2702 - f1-score: 0.1865 - val_loss: 0.8723 - val_accuracy: 0.3660 - val_f1-score: 0.1277
Epoch 30/400
74/74 [==============================] - 29s 393ms/step - loss: 0.8158 - accuracy: 0.2707 - f1-score: 0.1842 - val_loss: 0.8688 - val_accuracy: 0.3372 - val_f1-score: 0.1312
Epoch 31/400
74/74 [==============================] - 29s 387ms/step - loss: 0.8133 - accuracy: 0.2728 - f1-score: 0.1867 - val_loss: 0.8818 - val_accuracy: 0.3643 - val_f1-score: 0.1182
Epoch 32/400
74/74 [==============================] - 28s 383ms/step - loss: 0

Epoch 71/400
74/74 [==============================] - 29s 392ms/step - loss: 0.7752 - accuracy: 0.3142 - f1-score: 0.2248 - val_loss: 0.8902 - val_accuracy: 0.2263 - val_f1-score: 0.1098
Epoch 72/400
74/74 [==============================] - 29s 386ms/step - loss: 0.7730 - accuracy: 0.3167 - f1-score: 0.2270 - val_loss: 0.8664 - val_accuracy: 0.2785 - val_f1-score: 0.1336


In [ ]:
import keras.backend as K
def dice_coef(smooth=1e-6):
    def dice(y_true, y_pred):
        y_true_f = K.flatten(y_true)
        y_pred_f = K.flatten(y_pred)
        #print((y_true_f * y_pred_f).shape)
        intersection = K.sum(y_true_f * y_pred_f)
        return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return dice

#def bincross()

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef([1] * 11)(y_true, y_pred)
import tensorflow_addons as tfa
import sklearn.metrics
with strategy.scope():
    w = np.zeros((12))
    w[1:] = 1 / 11
    f1_score = sm.metrics.FScore()
    print(f1_score)
    dicel = sm.losses.DiceLoss()
    loss = dicel
    import sklearn
    import tensorflow_addons as tfa
    losses = {'jaccard_loss' : sm.losses.jaccard_loss, 'dice_loss' : sm.losses.dice_loss, 'focal_loss' : sm.losses.categorical_focal_loss, 'crossentropy' : sm.losses.categorical_crossentropy, 'cce_dice_loss' : sm.losses.cce_dice_loss, 'bce_jaccard_loss' : sm.losses.cce_jaccard_loss, 'focal_dice_loss' : sm.losses.categorical_focal_dice_loss, 'focal_jaccard_loss' : sm.losses.categorical_focal_jaccard_loss}
    size = 512
    s = res_s
    with strategy.scope():
        #f1_score = tfa.metrics.F1Score(num_classes=13, threshold=0.5, average = 'weighted')
        model = models[res](classes = 12, activation = 'softmax')
        model.compile(optimizer = 'rmsprop', loss = dice_coef_loss, metrics = ['accuracy', f1_score])
    print('*'*30)
    print('Fitting model ' + model_str + '...')
    print('*'*30)

    train = tf.data.Dataset.from_generator(train_data(s), output_signature=(
             tf.TensorSpec(shape=(512, 512, 3), dtype=tf.float16),
             tf.TensorSpec(shape=(512, 512, 12), dtype=tf.float16))).batch(3)

    valid = tf.data.Dataset.from_generator(valid_data(s), output_signature=(
             tf.TensorSpec(shape=(512, 512, 3), dtype=tf.float16),
             tf.TensorSpec(shape=(512, 512, 12), dtype=tf.float16))).batch(3)

    epochs = 400
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=70)
    model_checkpoint = ModelCheckpoint(f'weights_final1_master_{epochs}epochs.tf', monitor='val_accuracy', save_best_only=True, mode = 'max')

    history =  model.fit(train, epochs=epochs, verbose=1,
              validation_data = valid,
              callbacks=[model_checkpoint, early_stopping])


******************************
Fitting model Linknet...
******************************
Epoch 1/400
    246/Unknown - 180s 642ms/step - loss: 0.4807 - accuracy: 0.5431 - f1-score: 0.0588

INFO:tensorflow:Assets written to: weights_final1_master_400epochs.tf\assets


INFO:tensorflow:Assets written to: weights_final1_master_400epochs.tf\assets


246/246 [==============================] - 213s 774ms/step - loss: 0.4807 - accuracy: 0.5431 - f1-score: 0.0588 - val_loss: 0.5017 - val_accuracy: 0.5018 - val_f1-score: 0.0456
Epoch 2/400
246/246 [==============================] - 171s 695ms/step - loss: 0.4022 - accuracy: 0.5979 - f1-score: 0.0555 - val_loss: 0.4983 - val_accuracy: 0.5018 - val_f1-score: 0.0460
Epoch 3/400
246/246 [==============================] - 172s 695ms/step - loss: 0.4021 - accuracy: 0.5979 - f1-score: 0.0669 - val_loss: 0.4983 - val_accuracy: 0.5018 - val_f1-score: 0.0542
Epoch 4/400
246/246 [==============================] - 172s 697ms/step - loss: 0.4021 - accuracy: 0.5979 - f1-score: 0.0887 - val_loss: 0.4983 - val_accuracy: 0.5018 - val_f1-score: 0.0803
Epoch 5/400
246/246 [==============================] - 173s 704ms/step - loss: 0.4021 - accuracy: 0.5979 - f1-score: 0.1059 - val_loss: 0.4983 - val_accuracy: 0.5018 - val_f1-score: 0.0578
Epoch 6/400
246/246 [==============================] - 174s 705ms/s

ДАЛЬШЕ ПРО ДОМА!!!!




In [ ]:
import numpy as np
from PIL import Image
print('*'*30)
print('Loading and preprocessing train data...')
print('*'*30)

def train_data(bs = 4, size = 512):
    def train():
        img_train = []
        mask_train = []
        img_train = np.empty((bs, 4086, 4086, 3), dtype = 'float16')
        mask_train = np.empty((bs,4086, 4086, 1), dtype = 'float16')
        path = 'D:/Misha/buildings/Belgorod'
        list = os.listdir(path + '/ofp_jpgs_resized/train')
        listimg = list
        l1 = np.random.permutation(listimg)
        for j in range(len(l1)):
            #for k in range(bs):
                i = l1[j]
                img = Image.open(os.path.join(path, 'ofp_jpgs_resized/train', i))
                if size != 512:
                    img = img.resize((4608, 4608))
                img = np.asarray(img)
                img_train = img.astype('float16') / 255
                img = Image.open(os.path.join(path, 'buildings/train', i))
                if size != 512:
                    img = img.resize((4608, 4608))
                mask_train = np.asarray(img).astype('float16').reshape((img.size[0], img.size[1], 1))
                for i in range(img_train.shape[0] // size):
                    for j in range(img_train.shape[1] // size):
                        #mask_train = 1 - mask_train
                        #print(m1.shape)
                        #if np.max(mask_train) != 0:
                        yield tf.convert_to_tensor(img_train[i * size : (i + 1) * size, j* size : (j + 1) * size, :]), tf.convert_to_tensor(mask_train[i * size : (i + 1) * size, j* size : (j + 1) * size, :])
                img.close()
                #yield tf.convert_to_tensor(img_train), tf.convert_to_tensor(mask_train[:, :, 0])
    return train

def valid_data(bs = 4, size = 512):
    def valid():
        img_train = []
        mask_train = []
        img_train = np.empty((bs, 4086, 4086, 3), dtype = 'float16')
        mask_train = np.empty((bs,4086, 4086, 1), dtype = 'float16')
        path = 'D:/Misha/buildings/new_dataset/AerialImageDataset'
        list = os.listdir(path + '/valid/images')
        listimg = list
        l1 = np.random.permutation(listimg)
        for j in range(len(l1)):
            #for k in range(bs):
                i = l1[j]
                img = Image.open(os.path.join(path, 'valid/images', i))
                if size != 512:
                    img = img.resize((4608, 4608))
                img = np.asarray(img)
                img_train = img.astype('float16') / 255
                img = Image.open(os.path.join(path, 'valid/masks', i))
                if size != 512:
                    img = img.resize((4608, 4608))
                mask_train = np.asarray(img).astype('float16').reshape((img.size[0], img.size[1], 1)) / 255
                for i in range(img_train.shape[0] // size):
                    for j in range(img_train.shape[1] // size):
                        #mask_train = 1 - mask_train
                        #print(m1.shape)
                        #if np.max(mask_train) != 0:
                        yield tf.convert_to_tensor(img_train[i * size : (i + 1) * size, j* size : (j + 1) * size, :]), tf.convert_to_tensor(mask_train[i * size : (i + 1) * size, j* size : (j + 1) * size, :])

                img.close()
                #yield tf.convert_to_tensor(img_train), tf.convert_to_tensor(mask_train)
    return valid
def test_data(path, size = 1024):
    def test():
        bs = 4
        img_train = []
        mask_train = []
        img_train = np.empty((bs, 512, 512, 3), dtype = 'float16')
        mask_train = np.empty((bs,512, 512, 1), dtype = 'float16')
        #path = 'D:/Misha/buildings/new_dataset/AerialImageDataset'
        list = os.listdir(path + '/images')
        listimg = list
        l1 = list
        for j in range(len(l1)):
            #for k in range(bs):
                i = l1[j]
                img = Image.open(os.path.join(path, 'images', i)).resize((size, size))
                img_train = np.asarray(img).astype('float16') / 255

                img = Image.open(os.path.join(path, 'images', i)).resize((size, size))
                mask_train = np.asarray(img).astype('float16')
                #mask_train = 1 - mask_train
                img.close()
                #print(m1.shape)
                if np.max(mask_train) != 0:
                    yield tf.convert_to_tensor(img_train), tf.convert_to_tensor(mask_train[:, :, 0])
                #yield tf.convert_to_tensor(img_train), tf.convert_to_tensor(mask_train)
    return test

print('*'*30)
print('Creating and compiling model...')
print('*'*30)
strategy = tf.distribute.MirroredStrategy(cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

******************************
Loading and preprocessing train data...
******************************
******************************
Creating and compiling model...
******************************
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
Number of devices: 2


In [ ]:
res = 'Linknet'


In [ ]:
losses = {'jaccard_loss' : sm.losses.jaccard_loss, 'dice_loss' : sm.losses.dice_loss, 'binary_focal_loss' : sm.losses.binary_focal_loss, 'binary_crossentropy' : sm.losses.binary_crossentropy, 'bce_dice_loss' : sm.losses.bce_dice_loss, 'bce_jaccard_loss' : sm.losses.bce_jaccard_loss, 'binary_focal_dice_loss' : sm.losses.binary_focal_dice_loss, 'binary_focal_jaccard_loss' : sm.losses.binary_focal_jaccard_loss}

models = {'Linknet':sm.Linknet, 'FPN':sm.FPN, 'Unet': sm.Unet}
val_f1 = {'jaccard_loss':0, 'dice_loss':0, 'binary_focal_loss':0, 'binary_crossentropy':0, 'bce_dice_loss':0, 'bce_jaccard_loss':0, 'binary_focal_dice_loss':0, 'binary_focal_jaccard_loss':0}
for loss in losses:
    model_str = res
    print(model_str)
    with strategy.scope():
        model = models[model_str](classes = 1, activation = 'sigmoid')
        model.compile(optimizer = 'adam', loss = losses[loss], metrics = ['accuracy', sm.metrics.f1_score])
    print('*'*30)
    print('Fitting model ' + model_str + ' with ' + loss + '...')
    print('*'*30)

    train = tf.data.Dataset.from_generator(train_data(size = 512), output_signature=(
             tf.TensorSpec(shape=(512, 512, 3), dtype=tf.float16),
             tf.TensorSpec(shape=(512, 512, 1), dtype=tf.float16))).batch(3)

    valid = tf.data.Dataset.from_generator(valid_data(size = 512), output_signature=(
             tf.TensorSpec(shape=(512, 512, 3), dtype=tf.float16),
             tf.TensorSpec(shape=(512, 512, 1), dtype=tf.float16))).batch(3)

    epochs = 3
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_f1-score', patience=25)
    model_checkpoint = ModelCheckpoint(f'weights_' + model_str + f'_buildings_{epochs}epochs.tf', monitor='val_f1-score', save_best_only=True, mode = 'max')

    history =  model.fit(train, epochs=epochs, verbose=1,
              validation_data = valid,
              callbacks=[model_checkpoint, early_stopping])
    val_f1[loss] = np.max(np.asarray(history.history['val_f1-score']))
    path = 'D:/Misha/buildings/new_dataset/AerialImageDataset/test'
    test = tf.data.Dataset.from_generator(test_data(path), output_signature=(
             tf.TensorSpec(shape=(1024, 1024, 3), dtype=tf.float16),
             tf.TensorSpec(shape=(1024, 1024), dtype=tf.float16))).batch(1)
    res1 = model.predict(test)
    list = os.listdir(path + '/images')
    for i in range(len(list)):
        im = Image.open(os.path.join(path, 'images', list[i])).resize((1024, 1024))
        im = np.asarray(im) / 255
        im1 = np.empty(im.shape)
        for j in range(3):
            #print(np.multiply(im[:, :, j], res[i, :, :, 0]).shape)
            im1[:, :, j] = np.multiply(im[:, :, j], res1[i, :, :, 0])
        #print(np.max(im1))
        x = cv2.imwrite(os.path.join(path, 'predicted', list[i][:-4] + '_' + model_str + '_' + loss + '.jpg'), (im1 * 255) )
    path_mar = 'D:/Misha/buildings/Mariupol/datasets/images/train'
    list = os.listdir(path_mar + '/images')
    test = tf.data.Dataset.from_generator(test_data(path_mar), output_signature=(
             tf.TensorSpec(shape=(1024, 1024, 3), dtype=tf.float16),
             tf.TensorSpec(shape=(1024, 1024), dtype=tf.float16))).batch(1)

    res1 = model.predict(test)
    for j in range(len(list)):
        i = list[j]
        im = Image.open(os.path.join(path_mar, 'images', list[j])).resize((1024, 1024))
        im = np.asarray(im) / 255
        im1 = np.empty(im.shape)
        for k in range(3):
            im1[:, :, k] = im[:, :, k] * res1[j, :, :, 0]
        x = cv2.imwrite(os.path.join(path_mar, 'predicted', i[:-4] + '_' + model_str + '.jpg'), (im1 * 255))
        #print(x)

Linknet
******************************
Fitting model Linknet with jaccard_loss...
******************************
Epoch 1/3
INFO:tensorflow:batch_all_reduce: 79 all-reduces with algorithm = hierarchical_copy, num_packs = 1


INFO:tensorflow:batch_all_reduce: 79 all-reduces with algorithm = hierarchical_copy, num_packs = 1


INFO:tensorflow:batch_all_reduce: 79 all-reduces with algorithm = hierarchical_copy, num_packs = 1


INFO:tensorflow:batch_all_reduce: 79 all-reduces with algorithm = hierarchical_copy, num_packs = 1


   2880/Unknown - 1050s 360ms/step - loss: 0.7058 - accuracy: 0.8133 - f1-score: 0.4112

...

INFO:tensorflow:Assets written to: weights_Linknet_buildings_3epochs.tf\assets


2880/2880 [==============================] - 1152s 399ms/step - loss: 0.7350 - accuracy: 0.8920 - f1-score: 0.4759 - val_loss: 0.8067 - val_accuracy: 0.8678 - val_f1-score: 0.4714
Epoch 3/3
2880/2880 [==============================] - ETA: 0s - loss: 0.6687 - accuracy: 0.9059 - f1-score: 0.5360

INFO:tensorflow:Assets written to: weights_Linknet_buildings_3epochs.tf\assets


INFO:tensorflow:Assets written to: weights_Linknet_buildings_3epochs.tf\assets


537/537 [==============================] - 488s 907ms/step


In [ ]:
res = 'dice_loss'
"""for key in val_f1:
    if val_f1[res] < val_f1[key]:
        res = key
print(res)"""
print(val_f1)

NameError: name 'val_f1' is not defined

In [ ]:
    with mirrored_strategy.scope():
        model = models[model_str](classes = 1, activation = 'sigmoid')
    loss = res
    print(loss)
    model.compile(optimizer = 'adadelta', loss = losses[loss], metrics = ['accuracy', sm.metrics.f1_score])
    print('*'*30)
    print('Fitting model ' + model_str + '...')
    print('*'*30)

    train = tf.data.Dataset.from_generator(train_data(size = 512), output_signature=(
             tf.TensorSpec(shape=(512, 512, 3), dtype=tf.float16),
             tf.TensorSpec(shape=(512, 512, 1), dtype=tf.float16))).batch(4)

    valid = tf.data.Dataset.from_generator(valid_data(size = 512), output_signature=(
             tf.TensorSpec(shape=(512, 512, 3), dtype=tf.float16),
             tf.TensorSpec(shape=(512, 512, 1), dtype=tf.float16))).batch(4)

    epochs = 500
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_f1-score', patience=50, mode = 'max')
    model_checkpoint = ModelCheckpoint(f'weights_{model_str}_buildings_{epochs}epochs.tf', monitor='val_f1-score', save_best_only=True, mode = 'max')

    history =  model.fit(train, epochs=epochs, verbose=1,
              validation_data = valid,
              callbacks=[model_checkpoint, early_stopping])
    #val_f1[loss] = np.max(np.asarray(history.history['val_f1-score']))
    path = 'D:/Misha/buildings/new_dataset/AerialImageDataset/test'
    test = tf.data.Dataset.from_generator(test_data(path), output_signature=(
             tf.TensorSpec(shape=(1024, 1024, 3), dtype=tf.float16),
             tf.TensorSpec(shape=(1024, 1024), dtype=tf.float16))).batch(1)
    res = model.predict(test)
    list = os.listdir(path + '/images')
    for i in range(len(list)):
        im = Image.open(os.path.join(path, 'images', list[i])).resize((1024, 1024))
        im = np.asarray(im) / 255
        im1 = np.empty(im.shape)
        for j in range(3):
            #print(np.multiply(im[:, :, j], res[i, :, :, 0]).shape)
            im1[:, :, j] = np.multiply(im[:, :, j], res[i, :, :, 0])
        #print(np.max(im1))
        x = cv2.imwrite(os.path.join(path, 'predicted', list[i][:-4] + '_' + model_str + '.jpg'), (im1 * 255) )
    path_mar = 'D:/Misha/buildings/Mariupol/datasets/images/train'
    list = os.listdir(path_mar)
    test = tf.data.Dataset.from_generator(test_data(path), output_signature=(
             tf.TensorSpec(shape=(1024, 1024, 3), dtype=tf.float16),
             tf.TensorSpec(shape=(1024, 1024), dtype=tf.float16))).batch(1)

    res = model.predict(test)
    for j in range(len(list)):
        i = list[j]
        im = Image.open(os.path.join(path, 'images', list[j])).resize((1024, 1024))
        im = np.asarray(im) / 255
        im1 = np.empty(im.shape)
        for k in range(3):
            im1[:, :, k] = im[:, :, k] * res[j, :, :, 0]
        x = cv2.imwrite(os.path.join(path_mar, 'predicted', i[:-4] + '_' + model_str + '.jpg'), (im1 * 255))
        #print(x)


dice_loss
******************************
Fitting model Linknet...
******************************
Epoch 1/500
INFO:tensorflow:batch_all_reduce: 79 all-reduces with algorithm = hierarchical_copy, num_packs = 1
INFO:tensorflow:batch_all_reduce: 79 all-reduces with algorithm = hierarchical_copy, num_packs = 1
   2160/Unknown - 843s 383ms/step - loss: 0.8049 - accuracy: 0.6982 - f1-score: 0.1951

INFO:tensorflow:Assets written to: weights_Linknet_buildings_500epochs.tf\assets


INFO:tensorflow:Assets written to: weights_Linknet_buildings_500epochs.tf\assets


2160/2160 [==============================] - 985s 448ms/step - loss: 0.8049 - accuracy: 0.6982 - f1-score: 0.1951 - val_loss: 0.7467 - val_accuracy: 0.6642 - val_f1-score: 0.2533
Epoch 2/500
2160/2160 [==============================] - ETA: 0s - loss: 0.7687 - accuracy: 0.6845 - f1-score: 0.2313

INFO:tensorflow:Assets written to: weights_Linknet_buildings_500epochs.tf\assets


...

2160/2160 [==============================] - 923s 427ms/step - loss: 0.3345 - accuracy: 0.9249 - f1-score: 0.6655 - val_loss: 0.3016 - val_accuracy: 0.9170 - val_f1-score: 0.6984
Epoch 88/500
2160/2160 [==============================] - 910s 421ms/step - loss: 0.3333 - accuracy: 0.9251 - f1-score: 0.6667 - val_loss: 0.3053 - val_accuracy: 0.9121 - val_f1-score: 0.6947
Epoch 89/500
2160/2160 [==============================] - 908s 420ms/step - loss: 0.3319 - accuracy: 0.9253 - f1-score: 0.6681 - val_loss: 0.3221 - val_accuracy: 0.9173 - val_f1-score: 0.6779
Epoch 90/500
2160/2160 [==============================] - 914s 423ms/step - loss: 0.3306 - accuracy: 0.9256 - f1-score: 0.6694 - val_loss: 0.3074 - val_accuracy: 0.9131 - val_f1-score: 0.6926
Epoch 91/500
2160/2160 [==============================] - 919s 425ms/step - loss: 0.3291 - accuracy: 0.9258 - f1-score: 0.6709 - val_loss: 0.3108 - val_accuracy: 0.9169 - val_f1-score: 0.6892
Epoch 92/500
2160/2160 [=============================

INFO:tensorflow:Assets written to: weights_Linknet_buildings_500epochs.tf\assets


INFO:tensorflow:Assets written to: weights_Linknet_buildings_500epochs.tf\assets


2160/2160 [==============================] - 928s 429ms/step - loss: 0.3275 - accuracy: 0.9259 - f1-score: 0.6725 - val_loss: 0.2991 - val_accuracy: 0.9163 - val_f1-score: 0.7009
Epoch 93/500
2160/2160 [==============================] - 921s 426ms/step - loss: 0.3263 - accuracy: 0.9264 - f1-score: 0.6737 - val_loss: 0.3272 - val_accuracy: 0.9164 - val_f1-score: 0.6728
Epoch 94/500
2160/2160 [==============================] - 929s 430ms/step - loss: 0.3249 - accuracy: 0.9263 - f1-score: 0.6751 - val_loss: 0.3032 - val_accuracy: 0.9158 - val_f1-score: 0.6968
Epoch 95/500
2160/2160 [==============================] - 928s 429ms/step - loss: 0.3238 - accuracy: 0.9264 - f1-score: 0.6762 - val_loss: 0.3287 - val_accuracy: 0.9152 - val_f1-score: 0.6713
Epoch 96/500
2160/2160 [==============================] - 920s 426ms/step - loss: 0.3224 - accuracy: 0.9267 - f1-score: 0.6776 - val_loss: 0.3093 - val_accuracy: 0.9199 - val_f1-score: 0.6907
Epoch 97/500
2160/2160 [=============================

KeyboardInterrupt: 

In [ ]:
    with mirrored_strategy.scope():
        x = {'focal_dice_loss' : sm.losses.categorical_focal_dice_loss, 'f1_score' : sm.metrics.f1_score}
        model = keras.models.load_model('weights_Linknet_buildings_500epochs.tf', custom_objects = x, compile = False )
        model.compile(optimizer = 'adadelta', loss = sm.losses.dice_loss, metrics = ['accuracy', sm.metrics.f1_score])
    print('*'*30)
    model_str = 'Linknet'
    print('Fitting model ' + 'Linknet' + '...')
    print('*'*30)

    train = tf.data.Dataset.from_generator(train_data(size = 512), output_signature=(
             tf.TensorSpec(shape=(512, 512, 3), dtype=tf.float16),
             tf.TensorSpec(shape=(512, 512, 1), dtype=tf.float16))).batch(4)

    valid = tf.data.Dataset.from_generator(valid_data(size = 512), output_signature=(
             tf.TensorSpec(shape=(512, 512, 3), dtype=tf.float16),
             tf.TensorSpec(shape=(512, 512, 1), dtype=tf.float16))).batch(4)

    epochs = 400
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_f1-score', patience=20, mode = 'max')
    model_checkpoint = ModelCheckpoint(f'weights_{model_str}_buildings_500epochs.tf', monitor='val_f1-score', save_best_only=True, mode = 'max')

    history =  model.fit(train, epochs=epochs, verbose=1,
              validation_data = valid,
              callbacks=[model_checkpoint, early_stopping])

******************************
Fitting model Linknet...
******************************
Epoch 1/400
INFO:tensorflow:batch_all_reduce: 79 all-reduces with algorithm = hierarchical_copy, num_packs = 1
INFO:tensorflow:batch_all_reduce: 79 all-reduces with algorithm = hierarchical_copy, num_packs = 1


In [ ]:
    with mirrored_strategy.scope():
        model = models['Unet'](classes = 1, activation = 'sigmoid')
    loss = res
    print(loss)
    model.compile(optimizer = 'adadelta', loss = losses[loss], metrics = ['accuracy', sm.metrics.f1_score])
    print('*'*30)
    print('Fitting model ' + model_str + '...')
    print('*'*30)

    train = tf.data.Dataset.from_generator(train_data(size = 512), output_signature=(
             tf.TensorSpec(shape=(512, 512, 3), dtype=tf.float16),
             tf.TensorSpec(shape=(512, 512, 1), dtype=tf.float16))).batch(4)

    valid = tf.data.Dataset.from_generator(valid_data(size = 512), output_signature=(
             tf.TensorSpec(shape=(512, 512, 3), dtype=tf.float16),
             tf.TensorSpec(shape=(512, 512, 1), dtype=tf.float16))).batch(4)

    epochs = 500
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_f1-score', patience=50, mode = 'max')
    model_checkpoint = ModelCheckpoint(f'weights_' + model_str + f'_buildings_{epochs}epochs.tf', monitor='val_f1-score', save_best_only=True, mode = 'max')

    history =  model.fit(train, epochs=epochs, verbose=1,
              validation_data = valid,
              callbacks=[model_checkpoint, early_stopping])
    #val_f1[loss] = np.max(np.asarray(history.history['val_f1-score']))
    path = 'D:/Misha/buildings/new_dataset/AerialImageDataset/test'
    test = tf.data.Dataset.from_generator(test_data(path), output_signature=(
             tf.TensorSpec(shape=(1024, 1024, 3), dtype=tf.float16),
             tf.TensorSpec(shape=(1024, 1024), dtype=tf.float16))).batch(1)
    res = model.predict(test)
    list = os.listdir(path + '/images')
    for i in range(len(list)):
        im = Image.open(os.path.join(path, 'images', list[i])).resize((1024, 1024))
        im = np.asarray(im) / 255
        im1 = np.empty(im.shape)
        for j in range(3):
            #print(np.multiply(im[:, :, j], res[i, :, :, 0]).shape)
            im1[:, :, j] = np.multiply(im[:, :, j], res[i, :, :, 0])
        #print(np.max(im1))
        x = cv2.imwrite(os.path.join(path, 'predicted', list[i][:-4] + '_' + model_str + '.jpg'), (im1 * 255) )
    path_mar = 'D:/Misha/buildings/Mariupol/datasets/images/train'
    list = os.listdir(path_mar)
    test = tf.data.Dataset.from_generator(test_data(path), output_signature=(
             tf.TensorSpec(shape=(1024, 1024, 3), dtype=tf.float16),
             tf.TensorSpec(shape=(1024, 1024), dtype=tf.float16))).batch(1)

    res = model.predict(test)
    for j in range(len(list)):
        i = list[j]
        im = Image.open(os.path.join(path, 'images', list[j])).resize((1024, 1024))
        im = np.asarray(im) / 255
        im1 = np.empty(im.shape)
        for k in range(3):
            im1[:, :, k] = im[:, :, k] * res[j, :, :, 0]
        x = cv2.imwrite(os.path.join(path_mar, 'predicted', i[:-4] + '_' + model_str + '.jpg'), (im1 * 255))
        #print(x)


In [ ]:
import os
path = 'D:/Misha/buildings/Mariupol/datasets/images/'
for j in ('train', 'test', 'val'):
    path1 = os.path.join(path, j, 'images')
    list = os.listdir(path1)
    print(list1)
    for i in range(len(list)):
        os.rename(os.path.join(path1, list[i]), os.path.join(path1, list[i][37:] + '.jpg'))
        #print(os.path.join(path1, i[:-8] + i[:-4]))

In [ ]:
import numpy as np
from PIL import Image
print('*'*30)
print('Loading and preprocessing train data...')
print('*'*30)

def train_data(data):
    def train():
        path = 'D:/Misha/master/new/' + data
        list = os.listdir(path + '/train')
        listimg = list
        l1 = np.random.permutation(listimg)
        for j in range(len(l1)):
            #for k in range(bs):
                i = l1[j]
                #print(path)
                img = Image.open(os.path.join(path, 'train', i))
                img = img.resize((2048, 2048))
                img_train = np.asarray(img).astype('float16') / 255

                img = Image.open(os.path.join('D:/Misha/master/new/labels/train', i))
                img = img.resize((2048, 2048))
                mask_train = np.asarray(img).astype('float16')
                for i in range((mask_train.shape[0] + 511) // 512):
                    for j in range((mask_train.shape[1] + 511) // 512):
                        mask_train1 = mask_train[i * 512 : (i + 1) * 512, j * 512 : (j + 1) * 512]
                        img_train1 = img_train[i * 512 : (i + 1) * 512, j * 512 : (j + 1) * 512]
                        m1 = np.zeros((512, 512, 12))
                        for k in range(0, 12):
                            if np.argwhere(mask_train1 == k).shape[0] != 0:
                                x = np.argwhere(mask_train1 == k)
                                m1[x[:, 0], x[:, 1] , k] = 1
                        #mask_train = 1 - mask_train
                        yield tf.convert_to_tensor(img_train1), tf.convert_to_tensor(m1)
                #yield tf.convert_to_tensor(img_train), tf.convert_to_tensor(mask_train)
    return train


def valid_data(data):
    def train():
        path = 'D:/Misha/master/new/' + data
        list = os.listdir(path + '/valid')
        listimg = list
        l1 = np.random.permutation(listimg)
        for j in range(len(l1)):
            #for k in range(bs):
                i = l1[j]
                img = Image.open(os.path.join(path, 'valid', i))
                img = img.resize((2048, 2048))
                img_train = np.asarray(img).astype('float16') / 255

                img = Image.open(os.path.join('D:/Misha/master/new/labels/valid', i))
                img = img.resize((2048, 2048))
                mask_train = np.asarray(img).astype('float16')
                for i in range((mask_train.shape[0] + 511) // 512):
                    for j in range((mask_train.shape[1] + 511) // 512):
                        mask_train1 = mask_train[i * 512 : (i + 1) * 512, j * 512 : (j + 1) * 512]
                        img_train1 = img_train[i * 512 : (i + 1) * 512, j * 512 : (j + 1) * 512]
                        m1 = np.zeros((512, 512, 12))
                        for k in range(0, 12):
                            if np.argwhere(mask_train1 == k).shape[0] != 0:
                                x = np.argwhere(mask_train1 == k)
                                m1[x[:, 0], x[:, 1] , k] = 1
                        #mask_train = 1 - mask_train
                        yield tf.convert_to_tensor(img_train1), tf.convert_to_tensor(m1)
                #yield tf.convert_to_tensor(img_train), tf.convert_to_tensor(mask_train)
    return train

def test_data(data):
    def train():
        path = 'D:/Misha/master/new/' + data
        list = os.listdir(path + '/test')
        l1 = list
        for j in range(len(l1)):
            #for k in range(bs):
                i = l1[j]
                img = Image.open(os.path.join(path, 'test', i))
                img = img.resize((1024, 1024))
                img_train = np.asarray(img).astype('float16') / 255

                img = Image.open(os.path.join('D:/Misha/master/new/labels/test', i))
                img = img.resize((1024, 1024))
                mask_train = np.asarray(img).astype('float16')
                for i in range((mask_train.shape[0] + 511) // 512):
                    for j in range((mask_train.shape[1] + 511) // 512):
                        mask_train1 = mask_train[i * 512 : (i + 1) * 512, j * 512 : (j + 1) * 512]
                        img_train1 = img_train[i * 512 : (i + 1) * 512, j * 512 : (j + 1) * 512]
                m1 = np.zeros((1024, 1024, 12))

                yield tf.convert_to_tensor(img_train), tf.convert_to_tensor(m1)
                #yield tf.convert_to_tensor(img_train), tf.convert_to_tensor(mask_train)
    return train

print('*'*30)
print('Creating and compiling model...')
print('*'*30)
strategy = tf.distribute.MirroredStrategy(cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))


******************************
Loading and preprocessing train data...
******************************
******************************
Creating and compiling model...
******************************
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


Number of devices: 2


In [ ]:
res = 'Linknet'
res_l = 'focal_dice_loss'

In [ ]:
    res_s = 'lanczos'
    size = 512
    s = res_s
    with strategy.scope():
        model = models[res](classes = 12, activation = 'softmax')
        model.compile(optimizer = 'adam', loss = loss, metrics = ['accuracy', f1_score])
    print('*'*30)
    print('Fitting model ' + model_str + '...')
    print('*'*30)

    train = tf.data.Dataset.from_generator(train_data(s), output_signature=(
             tf.TensorSpec(shape=(512, 512, 3), dtype=tf.float16),
             tf.TensorSpec(shape=(512, 512, 12), dtype=tf.float16))).batch(3)

    valid = tf.data.Dataset.from_generator(valid_data(s), output_signature=(
             tf.TensorSpec(shape=(512, 512, 3), dtype=tf.float16),
             tf.TensorSpec(shape=(512, 512, 12), dtype=tf.float16))).batch(3)

    epochs = 500
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_f1-score', patience=50)
    model_checkpoint = ModelCheckpoint(f'weights_final_master_lanczos_{epochs}epochs.tf', monitor='val_f1-score', save_best_only=True, mode = 'max')

    history =  model.fit(train, epochs=epochs, verbose=1,
              validation_data = valid,
              callbacks=[model_checkpoint, early_stopping])


******************************
Fitting model Linknet...
******************************
Epoch 1/500
INFO:tensorflow:batch_all_reduce: 79 all-reduces with algorithm = hierarchical_copy, num_packs = 1


INFO:tensorflow:batch_all_reduce: 79 all-reduces with algorithm = hierarchical_copy, num_packs = 1


INFO:tensorflow:batch_all_reduce: 79 all-reduces with algorithm = hierarchical_copy, num_packs = 1


INFO:tensorflow:batch_all_reduce: 79 all-reduces with algorithm = hierarchical_copy, num_packs = 1


    384/Unknown - 204s 502ms/step - loss: 0.9579 - accuracy: 0.7195 - f1-score: 0.0613

...

INFO:tensorflow:Assets written to: weights_final_master_lanczos_500epochs.tf\assets


384/384 [==============================] - 218s 568ms/step - loss: 0.9355 - accuracy: 0.7280 - f1-score: 0.0798 - val_loss: 0.9618 - val_accuracy: 0.4328 - val_f1-score: 0.0735
Epoch 28/500
384/384 [==============================] - 207s 537ms/step - loss: 0.9341 - accuracy: 0.7236 - f1-score: 0.0816 - val_loss: 1.0285 - val_accuracy: 0.0677 - val_f1-score: 0.0319
Epoch 29/500
384/384 [==============================] - 206s 536ms/step - loss: 0.9365 - accuracy: 0.7272 - f1-score: 0.0792 - val_loss: 0.9804 - val_accuracy: 0.5489 - val_f1-score: 0.0574
Epoch 30/500
384/384 [==============================] - 206s 537ms/step - loss: 0.9334 - accuracy: 0.7325 - f1-score: 0.0817 - val_loss: 0.9887 - val_accuracy: 0.2361 - val_f1-score: 0.0501
Epoch 31/500
384/384 [==============================] - 205s 534ms/step - loss: 0.9340 - accuracy: 0.7383 - f1-score: 0.0812 - val_loss: 0.9686 - val_accuracy: 0.4094 - val_f1-score: 0.0643
Epoch 32/500
384/384 [==============================] - 206s 53

INFO:tensorflow:Assets written to: weights_final_master_lanczos_500epochs.tf\assets


INFO:tensorflow:Assets written to: weights_final_master_lanczos_500epochs.tf\assets


384/384 [==============================] - 218s 568ms/step - loss: 0.9286 - accuracy: 0.7414 - f1-score: 0.0867 - val_loss: 0.9745 - val_accuracy: 0.4840 - val_f1-score: 0.0736
Epoch 55/500
384/384 [==============================] - 207s 539ms/step - loss: 0.9276 - accuracy: 0.7368 - f1-score: 0.0876 - val_loss: 0.9685 - val_accuracy: 0.5049 - val_f1-score: 0.0683
Epoch 56/500
384/384 [==============================] - 206s 536ms/step - loss: 0.9284 - accuracy: 0.7388 - f1-score: 0.0871 - val_loss: 0.9965 - val_accuracy: 0.2235 - val_f1-score: 0.0500
Epoch 57/500
384/384 [==============================] - 206s 536ms/step - loss: 0.9281 - accuracy: 0.7380 - f1-score: 0.0872 - val_loss: 0.9793 - val_accuracy: 0.3432 - val_f1-score: 0.0624
Epoch 58/500
384/384 [==============================] - 207s 538ms/step - loss: 0.9273 - accuracy: 0.7384 - f1-score: 0.0874 - val_loss: 0.9837 - val_accuracy: 0.5371 - val_f1-score: 0.0636
Epoch 59/500
384/384 [==============================] - 206s 53

In [ ]:
os.system("shutdown /s /t 1")

In [ ]:
import cv2
from keras.models import load_model
import numpy as np
import os
from PIL import Image

In [ ]:
path = 'home/u39710/master/2002-cropped-img.tif'
pc = os.path.join(path, 'cloud_n')
model = load_model('home/u39710/res_cloud_weights_Linknet_buildings_400epochs_var2.tf', compile = False)
model.compile(optimizer = 'adam', loss = 'mse', metrics = ['accuracy'])
for i in ('train', 'test', 'valid'):
  p1 = os.path.join(path, i)
  l = os.listdir(p1)
  for j in l:
    img = np.array(Image.open(os.path.join(p1, j)))
    r1 = model.predict(np.array([img]))[0]
    x = cv2.imwrite(os.path.join(pc, j), r1 * 255)
    if not x:
      print(j)


1/1 [==============================] - 0s 21ms/step


In [ ]:
from PIL import Image

path = '/content/drive/MyDrive/master/2013-cropped-img.tif'
pc = os.path.join(path, 'cloud_n')
model = load_model('drive/MyDrive/res_weights_Linknet_master_600epochs_ver2.tf', compile = False)

model2 = load_model('drive/MyDrive/weights_with_clouds_1_master_800epochs_ver2.tf', compile = False)
model.compile(optimizer = 'adam', loss = 'mse', metrics = ['accuracy'])
model2.compile(optimizer = 'adam', loss = 'mse', metrics = ['accuracy'])
img = np.array(Image.open(path).resize((4096, 4096)))
print(img.shape)
r1 = model.predict(np.array([img[:, :, :3]]))[0]

r2 = model2.predict(np.array([img[:, :, :3]]))[0]
r1 = np.mean(np.array([r1, r2]), axis = 0)

OSError: ignored

In [ ]:
a = [0] * 9
a[0] = [0, 0, 0]
a[1] = [0, 0, 255]
a[2] = [0, 255, 0]
a[3] = [255, 0, 0]
a[4] = [255, 255, 0]
a[5] = [255, 0, 255]
a[6] = [0, 255, 255]
a[8] = [255, 255, 255]

In [ ]:
res1 = np.empty((4096, 4096, 3))
res = np.argmax(r1, axis = 2)


In [ ]:
print(np.max(res))
for i in range(9):
  res1[np.nonzero(res == i)] = a[i]
x = cv2.imwrite('2013_pred_new.jpg', res1)
if not x:
  print(0)

8


In [ ]:
from PIL import Image
import os
import numpy as np
import cv2
path = 'home/u39710/divided'
pzz = 'home/u39710/'
pc = os.path.join(path, 'cloud_n')
for i in ('test', 'valid'):
  print(i)
  p1 = os.path.join(path, i)
  l = os.listdir(p1)
  print(len(l))
  for j1 in range(len(l)):
    if j1 % 10 == 0:
      print(j1)
    j = l[j1]
    img = np.array(Image.open(os.path.join(pc, j)))
    img1 = np.array(Image.open(os.path.join(pzz, j)))
    img1[np.nonzero(img > 0.8)] = 8
    print(len(np.nonzero(img > 0.8)[0]))
    x = cv2.imwrite(os.path.join('home/u39710/PZZ_N', j), img1)
    if not x:
      print(j)

test
191
0
37005
12657
2535
43636
41872
303
4821
36450
11441
16088
10
30301
20178
21709
19718
21117
15474
18389
52457
4374
7128
20
16035
36306
12403
51155
51
12486
22257
7328
11117
55750
30
16875
20299
8400
64379
5758
5342
3686
36716
54001
10872
40
12716


KeyboardInterrupt: ignored

In [ ]:
import shutil
import os
path = '/content/drive/MyDrive/master/divided'
pzz = '/content/drive/MyDrive/master/PZZ'
pc = os.path.join(path, 'cloud_n')
shutil.copytree(path, 'divided')
import shutil
shutil.copytree('/content/drive/MyDrive/master/PZZ_N', 'divided/pzz')

'divided/pzz'

In [ ]:
pathpzz = '/content/drive/MyDrive/master/PZZ'
pathcl = '/content/drive/MyDrive/master/divided/cloud_n'
l = os.listdir(pathpzz)
p1 = 'pzzcl'
if not os.path.isdir(p1):
  os.mkdir(p1)
for i in l:
  img = np.array(Image.open(os.path.join(pathpzz, i)))
  img1 = np.array(Image.open(os.path.join(pathcl, i)))
  img[np.nonzero(img1 > 0.7)] = 8
  x = cv2.imwrite(os.path.join(p1, i), img)
  if not x:
    print(i)

In [ ]:
import tensorflow as tf
model = tf.keras.applications.ResNet50()
path = 'D:\\шад\\dataset'
import os
from PIL import Image
import numpy as np
l = os.listdir(path)
res = np.empty((len(l), 1000))
for i1 in range(len(l)):
    i = l[i1]
    if i1 % 100 == 0:
        print(i1)
    img = np.array(Image.open(os.path.join(path, l[0])).resize((224, 224)))
    res[i1] = model.predict(np.array([img]))[0]

In [ ]:
dist = np.empty((len(l), len(l)))
with open('res.txt', 'w') as f:
    for i in range(len(l)):
        dist = [9999999] * len(l)
        for j in range(len(i)):
            if j != i:
              dist[i] =

In [ ]:
a = np.array([[[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]], [[13, 14, 15, 16], [17, 18, 19, 20], [21, 22, 23, 24]]])
print(np.mean(a, axis = 0))

[[ 7.  8.  9. 10.]
 [11. 12. 13. 14.]
 [15. 16. 17. 18.]]
